# System Description

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.ticker import MaxNLocator
import random 
import re
import math
import seaborn as sns
from astropy.convolution import convolve
from astropy.convolution.kernels import Gaussian1DKernel
from astropy.convolution.kernels import Gaussian2DKernel
import os
from matplotlib.ticker import PercentFormatter


## Set parameters

### Set system parameters

#### Load entry record updating function

In [ ]:
def Update_dic_entry_record(dic_entry_record, T_total_temp=0.0, T_key_total_temp=0.0, T_react_w_total_temp=0.0, T_react_s_total_temp=0.0, T_bounded_l_total_temp = 0.0, T_bounded_w_total_temp=0.0, T_bounded_s_total_temp=0.0, T_error_total_temp=0.0, T_interrupt_total_temp=0.0, T_event_total_temp=0.0, N_bounded_l_total_temp=0, N_bounded_w_total_temp=0, N_bounded_s_total_temp=0, KS_total_temp=0, KS_type_total_temp=0, KS_select_total_temp=0, KS_bounded_l_total_temp=0, KS_bounded_w_total_temp=0, KS_bounded_s_total_temp=0, KS_error_total_temp=0, KS_interrup_total_temp=0):

    dic_entry_record['T_total'] += T_total_temp
    dic_entry_record['T_key_total'] += T_key_total_temp
    dic_entry_record['T_react_w_total'] += T_react_w_total_temp
    dic_entry_record['T_react_s_total'] += T_react_s_total_temp
    dic_entry_record['T_react_total'] += T_react_w_total_temp + T_react_s_total_temp
    dic_entry_record['T_bounded_l_total'] += T_bounded_l_total_temp
    dic_entry_record['T_bounded_w_total'] += T_bounded_w_total_temp
    dic_entry_record['T_bounded_s_total'] += T_bounded_s_total_temp

    dic_entry_record['T_error_total'] += T_error_total_temp
    dic_entry_record['T_interrupt_total'] += T_interrupt_total_temp
    dic_entry_record['T_event_total'] += T_event_total_temp
    dic_entry_record['N_bounded_l_total'] += N_bounded_l_total_temp
    dic_entry_record['N_bounded_w_total'] += N_bounded_w_total_temp
    dic_entry_record['N_bounded_s_total'] += N_bounded_s_total_temp
    dic_entry_record['N_bounded_total'] += N_bounded_l_total_temp + N_bounded_w_total_temp + N_bounded_s_total_temp
    dic_entry_record['KS_total'] += KS_total_temp
    dic_entry_record['KS_type_total'] += KS_type_total_temp
    dic_entry_record['KS_select_total'] += KS_select_total_temp
    dic_entry_record['KS_bounded_l_total'] += KS_bounded_l_total_temp
    dic_entry_record['KS_bounded_w_total'] += KS_bounded_w_total_temp
    dic_entry_record['KS_bounded_s_total'] += KS_bounded_s_total_temp
    dic_entry_record['KS_error_total'] += KS_error_total_temp
    dic_entry_record['KS_interrupt_total'] += KS_interrup_total_temp

    return dic_entry_record


#### System output controller

In [ ]:
BOOL_PRINT = False

### Set user parameters

#### User ability

In [ ]:
# T_key = 0.6 # redefine to suit AAC, time for press the key in seconds 
# T_react_w = 1.2 # redefine to suit AAC, time for cognitive reactions
# T_react_s = 5.42 # redefine to suit AAC, time for cognitive reactions
T_key = 0.26 # POK referred data
T_react_w = 0.45 # POK referred data
T_react_s = 1.87 

R_rational = 1 # if random.random() < R_rational, execute optimal action
R_error = 0 # if random.random() < R_error, action execution failure

R_interrupt = 0.0
T_event = 60*random.random()
def T_interrupt(T_event): # the probability of the system have correct sentence predictions
    a=0.189
    c=1.03
    return a*math.log(T_event)+c

## Surrogate user model

### Global Flags

### Check human factors

In [ ]:
def Record_extra_actions(dic_entry_record, dic_rational_record, T_extra_temp, KS_extra_temp):

    if dic_rational_record['BOOL_RECORD_ON']:
        if dic_rational_record['BOOL_RECORD_S_ON']:
            dic_entry_record['T_extra_s'] += T_extra_temp
            dic_entry_record['KS_extra_s'] += KS_extra_temp
        else:
            if dic_rational_record['BOOL_RECORD_W_ON']:
                dic_entry_record['T_extra_w'] += T_extra_temp
                dic_entry_record['KS_extra_w'] += KS_extra_temp
            else:
                if dic_rational_record['BOOL_RECORD_L_ON']:
                    dic_entry_record['T_extra_l'] += T_extra_temp
                    dic_entry_record['KS_extra_l'] += KS_extra_temp 

    dic_entry_record['T_extra_total'] = dic_entry_record['T_extra_s'] + dic_entry_record['T_extra_w'] + dic_entry_record['T_extra_l']
    dic_entry_record['KS_extra_total'] = dic_entry_record['KS_extra_s'] + dic_entry_record['KS_extra_w'] + dic_entry_record['KS_extra_l']
    

    return dic_entry_record

In [ ]:
def Record_time_saving_from_reactions(dic_entry_record, dic_rational_record, T_react_saving_temp):
    if dic_rational_record['BOOL_RECORD_ON']:
        if dic_rational_record['BOOL_RECORD_S_ON']:
            dic_entry_record['T_react_saving_s'] += T_react_saving_temp
        else:
            if dic_rational_record['BOOL_RECORD_W_ON']:
                dic_entry_record['T_react_saving_w'] += T_react_saving_temp
            else:
                if dic_rational_record['BOOL_RECORD_L_ON']:
                    dic_entry_record['T_react_saving_l'] += T_react_saving_temp

    dic_entry_record['T_react_saving_total'] = dic_entry_record['T_react_saving_s'] + dic_entry_record['T_react_saving_w'] + dic_entry_record['T_react_saving_l']
    return dic_entry_record


In [ ]:
def Check_human_factors(dic_entry_record, dic_rational_record, FLAG_TASK, current_letter=None, letter_index=None, current_word=None, word_index=None, current_word_list=None):    
    
    R_rational = dic_entry_record['R_rational']
    R_error = dic_entry_record['R_error']
    # R_interrupt = dic_entry_record['R_interrupt']

    # L_min_w = dic_entry_record['L_min_w']
    # k_look_w = dic_entry_record['k_look_w']
    # p_max_w = dic_entry_record['p_max_w']
    # L_min_s = dic_entry_record['L_min_s']
    # k_look_s = dic_entry_record['k_look_s']
    # p_max_s = dic_entry_record['p_max_s']

    T_human_factors = 0
    KS_human_factors = 0
    BOOL_RATIONAL_NEXT = True
    # check if the action is rational 


            

    rand = random.random()

    if rand < R_rational:
        # it is rational
        BOOL_RATIONAL_NEXT = True
        # print(f"Rational, R_rational = {R_rational}")
    else:
        # it is irrational
        BOOL_RATIONAL_NEXT = False
        # print(f"Irrational, R_rational = {R_rational}")

    

    # check if it is an action execution failure
    if random.random() < R_error:
        # it is a failure, need correction operations
        # press a wrong key
        if BOOL_PRINT:
            print("* Press a wrong letter")
        dic_entry_record = Update_dic_entry_record(dic_entry_record, T_total_temp=T_key, T_key_total_temp=T_key, T_error_total_temp=T_key, KS_total_temp=1, KS_type_total_temp=1, KS_error_total_temp=1)
        T_human_factors += T_key
        KS_human_factors += 1
        if FLAG_TASK=='LETTER' and current_letter != None:
            # delete wrong letter
            dic_entry_record = Update_dic_entry_record(dic_entry_record, T_total_temp=T_key, T_key_total_temp=T_key, T_error_total_temp=T_key, KS_total_temp=1, KS_type_total_temp=1, KS_error_total_temp=1)
            T_human_factors += T_key
            KS_human_factors += 1
            if BOOL_PRINT:
                print(f"aim letter: {current_letter}")
                print("* Delete the wrong letter")
        elif FLAG_TASK=='WORD_PRED' and current_word != None:
            # select a wrong word, delect irrelvant letters
            KS_this_temp = len(current_word)-letter_index
            T_this_temp = T_key*KS_this_temp
            dic_entry_record = Update_dic_entry_record(dic_entry_record, T_total_temp=T_this_temp, T_key_total_temp=T_this_temp, T_error_total_temp=T_this_temp, KS_total_temp=KS_this_temp, KS_type_total_temp=KS_this_temp, KS_error_total_temp=KS_this_temp)
            T_human_factors += T_key*(len(current_word)-letter_index)
            KS_human_factors += len(current_word)-letter_index
            if BOOL_PRINT:
                print(f"** Delete {len(current_word)-letter_index} wrong letters")
        elif FLAG_TASK=='SENT_PRED' and current_word_list != None:
            # select a wrong sentence, delete irrelvant letters
            typed_char = ' '.join(current_word_list[:word_index])
            sentence = ' '.join(current_word_list)
            KS_this_temp = len(sentence)-len(typed_char)
            T_this_temp = T_key*KS_this_temp
            dic_entry_record = Update_dic_entry_record(dic_entry_record, T_total_temp=T_this_temp, T_key_total_temp=T_this_temp, T_error_total_temp=T_this_temp, KS_total_temp=KS_this_temp, KS_type_total_temp=KS_this_temp, KS_error_total_temp=KS_this_temp)
            T_human_factors += T_key*(len(sentence)-len(typed_char))
            KS_human_factors += len(sentence)-len(typed_char)
            if BOOL_PRINT:
                print(f"*** Delete {len(sentence)-len(typed_char)} wrong letters")
    else:
        #  it is not a failure
        pass


    return dic_entry_record, dic_rational_record, T_human_factors, KS_human_factors, BOOL_RATIONAL_NEXT

### Enter current word

In [ ]:
def Enter_current_word(dic_entry_record, dic_rational_record, word_list, word):

    L_min_w = dic_entry_record['L_min_w']
    k_look_w = dic_entry_record['k_look_w']
    p_max_w = dic_entry_record['p_max_w']


    next_word = ""

    if BOOL_PRINT:
        print(f"Entering current word: {word}")
    BOOL_TYPING = True
    BOOL_WORD_SELECTED = False
    T_current_word = 0
    KS_current_word = 0
    current_char_index = 0
    if len(word) < L_min_w:
        if BOOL_PRINT:
            print(f"Current word \"{word}\" is too short to use word pred")
        # type out all letters without prediction with the space
        # check human factors
        for i, letter in enumerate(word):
            dic_entry_record, dic_rational_record, T_temp, KS_temp, BOOL_RATIONAL_L_min_w_True = Check_human_factors(dic_entry_record, dic_rational_record, FLAG_TASK='LETTER', current_letter=letter) # BOOL_RATIONAL
            T_current_word += T_temp
            KS_current_word += KS_temp
            # press the expected key, same for rational and irrational
            if BOOL_RATIONAL_L_min_w_True==True or BOOL_RATIONAL_L_min_w_True==False:
                dic_entry_record = Update_dic_entry_record(dic_entry_record, T_total_temp=T_key, T_key_total_temp=T_key, KS_total_temp=1, KS_type_total_temp=1)
                T_current_word += T_key
                KS_current_word += 1
                current_char_index += 1
                if BOOL_PRINT:
                    print(f"-> KS_total={dic_entry_record['KS_total']}. Type letter: {letter}. Same operation for rational and irrational cases")
        # press space
        dic_entry_record = Update_dic_entry_record(dic_entry_record, T_total_temp=T_key, T_key_total_temp=T_key, KS_total_temp=1, KS_type_total_temp=1)
        T_current_word += T_key
        KS_current_word += 1
        
        if BOOL_PRINT:
            print(f"-> KS_total={dic_entry_record['KS_total']}. Press space to finish the word")
    else:
        if BOOL_PRINT:
            print(f"Current word \"{word}\" is long enough to use word pred")
        # could use prediction
        # check human factors before typing the first key
        dic_entry_record, dic_rational_record, T_temp, KS_temp, BOOL_RATIONAL_L_min_w_False = Check_human_factors(dic_entry_record, dic_rational_record, FLAG_TASK='LETTER',current_letter=word[current_char_index])
        T_current_word += T_temp
        KS_current_word += KS_temp
        # press the initial letter of the word
        if BOOL_RATIONAL_L_min_w_False==True or BOOL_RATIONAL_L_min_w_False==False:
            dic_entry_record = Update_dic_entry_record(dic_entry_record, T_total_temp=T_key, T_key_total_temp=T_key, KS_total_temp=1, KS_type_total_temp=1)
            T_current_word += T_key
            # current_char_index = 0
            KS_current_word += 1
            if BOOL_PRINT:
                print(f"-> KS_total={dic_entry_record['KS_total']}. Type the initial letter: {word[current_char_index]}. Same operation for rational and irrational cases")
            current_char_index += 1
        while BOOL_TYPING:
            # start from the second letter
            if current_char_index < len(word): 
                # not finish
                if current_char_index+1 > p_max_w:
                    # type out remainig letters in word
                    if BOOL_PRINT:
                        print("Word perseverance reached, type out remaining letters in the word")
                    for letter in word[current_char_index:]:
                        dic_entry_record, dic_rational_record, T_temp, KS_temp, BOOL_RATIONAL_p_max_w_True = Check_human_factors(dic_entry_record, dic_rational_record, FLAG_TASK='LETTER',current_letter=letter)
                        T_current_word += T_temp
                        KS_current_word += KS_temp
                        if BOOL_RATIONAL_p_max_w_True==True or BOOL_RATIONAL_p_max_w_True==False:
                            dic_entry_record = Update_dic_entry_record(dic_entry_record, T_total_temp=T_key, T_key_total_temp=T_key, KS_total_temp=1, KS_type_total_temp=1)
                            T_current_word += T_key
                            KS_current_word += 1
                            if BOOL_PRINT:
                                print(f"-> KS_total={dic_entry_record['KS_total']}. Type letter: {letter}. Same operation for rational and irrational cases")
                            current_char_index += 1
                    # Press space 
                    dic_entry_record = Update_dic_entry_record(dic_entry_record, T_total_temp=T_key, T_key_total_temp=T_key, KS_total_temp=1, KS_type_total_temp=1)
                    T_current_word += T_key
                    KS_current_word += 1
                    if BOOL_PRINT:
                        print(f"-> KS_total={dic_entry_record['KS_total']}. Press space to finish the word")

                    break
                else:
                    if BOOL_PRINT:
                        print("Haven't reached word perseverance yet")
                    if current_char_index+1 > k_look_w: # could start to check word pred
                        if BOOL_PRINT:
                            print("Reached word type-then-look, could use word pred")

                        dic_entry_record, dic_rational_record, T_temp, KS_temp, BOOL_RATIONAL_k_look_w_True = Check_human_factors(dic_entry_record, dic_rational_record, FLAG_TASK='WORD_PRED', letter_index=current_char_index, current_word=word)
                        T_current_word += T_temp
                        KS_current_word += KS_temp
                        if BOOL_RATIONAL_k_look_w_True:
                            if BOOL_PRINT:
                                print("A rational action, check current word pred")
                            dic_entry_record = Update_dic_entry_record(dic_entry_record, T_total_temp=T_react_w, T_react_w_total_temp=T_react_w)
                            T_current_word += T_react_w

                            if random.random() < P_pred_w_c:
                            # current word pred obtained
                            # obtain pred word
                                if BOOL_PRINT:
                                    print("Current word pred obtained.")
                                
                                # select word pred
                                dic_entry_record = Update_dic_entry_record(dic_entry_record, T_total_temp=T_key, T_key_total_temp=T_key, KS_total_temp=1, KS_select_total_temp=1)
                                T_current_word += T_key
                                KS_current_word += 1
                                current_char_index = len(word)-1
                                BOOL_WORD_SELECTED = True
                                if BOOL_PRINT:
                                    print(f"-> KS_total={dic_entry_record['KS_total']}. A rational action, select word {word}")
                                break
                            else:
                            # not obtain word pred, type next letter
                                BOOL_WORD_SELECTED = False
                                dic_entry_record, dic_rational_record, T_temp, KS_temp, BOOL_RATIONAL_P_pred_w_c_False = Check_human_factors(dic_entry_record, dic_rational_record, FLAG_TASK='LETTER', current_letter=word[current_char_index])
                                T_current_word += T_temp
                                KS_current_word += KS_temp
                                if BOOL_RATIONAL_P_pred_w_c_False==True or BOOL_RATIONAL_P_pred_w_c_False==False:
                                    dic_entry_record = Update_dic_entry_record(dic_entry_record, T_total_temp=T_key, T_key_total_temp=T_key, KS_total_temp=1, KS_type_total_temp=1)
                                    T_current_word += T_key
                                    KS_current_word += 1
                                    if BOOL_PRINT:
                                        print(f"-> KS_total={dic_entry_record['KS_total']}. Not obtain word pred, type next letter: {word[current_char_index]}. Same operation for rational and irrational cases")
                                    current_char_index += 1 

                                    if current_char_index == len(word):
                                        # press space
                                        dic_entry_record = Update_dic_entry_record(dic_entry_record, T_total_temp=T_key, T_key_total_temp=T_key, KS_total_temp=1, KS_type_total_temp=1)
                                        T_current_word += T_key
                                        KS_current_word += 1
                                        if BOOL_PRINT:
                                            print(f"-> KS_total={dic_entry_record['KS_total']}. Press space to finish the word")

                        else:
                            # Irrational, type next letter without checking the word prediction.
                            if BOOL_PRINT:
                                print(f"-> KS_total={dic_entry_record['KS_total']+1}. An irrational action: letter level. not check word pred, type next letter {word[current_char_index]}")
                            if dic_rational_record['BOOL_RECORD_S_ON'] == False and dic_rational_record['BOOL_RECORD_W_ON'] == False:
                                dic_rational_record['BOOL_RECORD_L_ON'] = True
                                dic_rational_record['BOOL_RECORD_ON'] = False
                            # print(f"BOOL_RECORD_S_ON: {dic_rational_record['BOOL_RECORD_S_ON']} - {dic_entry_record['KS_extra_s']}, BOOL_RECORD_W_ON: {dic_rational_record['BOOL_RECORD_W_ON']} - {dic_entry_record['KS_extra_w']}, BOOL_RECORD_L_ON: {dic_rational_record['BOOL_RECORD_L_ON']} - {dic_entry_record['KS_extra_l']}")

                            dic_entry_record = Update_dic_entry_record(dic_entry_record, T_total_temp=T_key, T_key_total_temp=T_key, N_bounded_l_total_temp=1, T_bounded_l_total_temp=T_key, KS_total_temp=1, KS_type_total_temp=1, KS_bounded_l_total_temp=1)
                            if dic_rational_record['BOOL_RECORD_S_ON'] == False and dic_rational_record['BOOL_RECORD_W_ON'] == False:
                                dic_rational_record['BOOL_RECORD_ON'] = True
                            if current_char_index+1 < len(word): 
                                # if the pred is last letter, then there is an action anyway - either click letter or choose pred, so no extra action
                                dic_entry_record = Record_extra_actions(dic_entry_record, dic_rational_record, T_extra_temp=T_key, KS_extra_temp=1)
                            T_current_word += T_key
                            KS_current_word += 1
                            # Update the time saving
                            dic_entry_record = Record_time_saving_from_reactions(dic_entry_record, dic_rational_record, T_react_saving_temp=T_react_w)
                            
                            current_char_index += 1
                            if dic_rational_record['BOOL_RECORD_S_ON'] == False and dic_rational_record['BOOL_RECORD_W_ON'] == False:
                                dic_rational_record['BOOL_RECORD_ON'] = False
                                dic_rational_record['BOOL_RECORD_L_ON'] = False
                            # print(f"BOOL_RECORD_S_ON: {dic_rational_record['BOOL_RECORD_S_ON']} - {dic_entry_record['KS_extra_s']}, BOOL_RECORD_W_ON: {dic_rational_record['BOOL_RECORD_W_ON']} - {dic_entry_record['KS_extra_w']}, BOOL_RECORD_L_ON: {dic_rational_record['BOOL_RECORD_L_ON']} - {dic_entry_record['KS_extra_l']}")

                            
                            if current_char_index == len(word):
                                # press space
                                dic_entry_record = Update_dic_entry_record(dic_entry_record, T_total_temp=T_key, T_key_total_temp=T_key, KS_total_temp=1, KS_type_total_temp=1)
                                T_current_word += T_key
                                KS_current_word += 1
                                if BOOL_PRINT:
                                    print(f"-> KS_total={dic_entry_record['KS_total']}. Press space to finish the word")

                    else:
                        # do not check word pred, just type next letter
                        dic_entry_record, dic_rational_record, T_temp, KS_temp, BOOL_RATIONAL_k_look_w_False = Check_human_factors(dic_entry_record, dic_rational_record, FLAG_TASK='LETTER', current_letter=word[current_char_index])
                        T_current_word += T_temp
                        KS_current_word += KS_temp
                        if BOOL_RATIONAL_k_look_w_False==True or BOOL_RATIONAL_k_look_w_False==False:
                            dic_entry_record = Update_dic_entry_record(dic_entry_record, T_total_temp=T_key, T_key_total_temp=T_key, KS_total_temp=1, KS_type_total_temp=1)
                            T_current_word += T_key
                            KS_current_word += 1
                            if BOOL_PRINT:
                                print(f"-> KS_total={dic_entry_record['KS_total']}. Not reach word type-then-look, just type next letter: {word[current_char_index]}. Same operation for rational and irrational cases")
                            current_char_index += 1
                            
                            if current_char_index == len(word):
                                # press space
                                dic_entry_record = Update_dic_entry_record(dic_entry_record, T_total_temp=T_key, T_key_total_temp=T_key, KS_total_temp=1, KS_type_total_temp=1)
                                T_current_word += T_key
                                KS_current_word += 1
                                if BOOL_PRINT:
                                    print(f"-> KS_total={dic_entry_record['KS_total']}. Press space to finish the word")
            else:
                # current word is finished
                if BOOL_PRINT:
                    print(f"Current word {word} is finished")
                BOOL_TYPING = False
    # Assign next word
    if word_list.index(word)+1 == len(word_list):
        next_word = ""
        if BOOL_WORD_SELECTED == False:
            # Remove the space after the final word
            if BOOL_PRINT:
                print("Last word typed, remove the space after the final word")
            T_current_word -= T_key
            KS_current_word -= 1
            # Special dic_entry_record
            dic_entry_record['T_total'] -= T_key
            dic_entry_record['T_key_total'] -= T_key
            dic_entry_record['KS_total'] -= 1
            dic_entry_record['KS_type_total'] -= 1
    else:
        next_word = word_list[word_list.index(word)+1]
                
    return dic_entry_record, dic_rational_record, next_word, T_current_word, KS_current_word, BOOL_WORD_SELECTED

### Check sentence finish

In [ ]:
def Check_sentence_finish(dic_entry_record, dic_rational_record, current_word, word_list):
    # Start from 2nd word

    L_min_s = dic_entry_record['L_min_s']
    k_look_s = dic_entry_record['k_look_s']
    p_max_s = dic_entry_record['p_max_s']

    T_sentence = 0
    KS_sentence = 0
    next_word = ''
    BOOL_SENTENCE_UNFINISH = True

    # if current_word == word_list[0]:
    #     if len(word_list) > 1:
    #         current_word = word_list[1]

    # find word selected, it will be typed again
    if current_word == word_list[-1]: #  and current_word != word_list[0]
        if BOOL_PRINT:
            print(">>> Sentence finished")
            print("------------------------------------------------------------")
        BOOL_SENTENCE_UNFINISH = False
    else:
        if len(word_list) < L_min_s:
            # Do not use sentence pred, Type next word. 
            if BOOL_PRINT:
                print("This sentence is too short to use sentence pred")
            dic_entry_record, dic_rational_record, T_temp, KS_temp, next_word, BOOL_SENTENCE_UNFINISH = Enter_next_word(dic_entry_record, dic_rational_record, current_word, word_list)
            T_sentence += T_temp
            KS_sentence += KS_temp
        else: 
            if BOOL_PRINT:
                print("This sentence is long enough to use sentence pred")
            current_word_index = Find_current_word_index(current_word, word_list)
            # this may differ from generation and retrieval
            if current_word_index+1 > p_max_s:
                # Do not use sentence pred, type next word out
                if BOOL_PRINT:
                    print("Sentence perseverance reached, not use sentence pred, type out next word in the sentence.")
                dic_entry_record, dic_rational_record, T_temp, KS_temp, next_word, BOOL_SENTENCE_UNFINISH = Enter_next_word(dic_entry_record, dic_rational_record, current_word, word_list)
                T_sentence += T_temp
                KS_sentence += KS_temp
            else: 
                if BOOL_PRINT:
                    print("Haven't reached sentence perseverance yet, could use sentence pred.")
                if current_word_index+1 > k_look_s:
                    if BOOL_PRINT:
                        print("Reached sentence type-then-look, could use sentence pred")
                    dic_entry_record, dic_rational_record, T_temp, KS_temp, BOOL_RATIONAL_k_look_s_True = Check_human_factors(dic_entry_record, dic_rational_record, FLAG_TASK='SENT_PRED', current_word=current_word, current_word_list=word_list)
                    T_sentence += T_temp
                    KS_sentence += KS_temp
                    if BOOL_RATIONAL_k_look_s_True:
                        dic_entry_record = Update_dic_entry_record(dic_entry_record, T_react_s_total_temp=T_react_s)        
                        T_sentence += T_react_s
                        if random.random() < P_pred_s:
                            if BOOL_PRINT:
                                print("Obtained sentence pred")
                            # obtained sentence pred and click to select 
                            # dic_entry_record, dic_rational_record, T_temp, KS_temp, BOOL_RATIONAL_P_pred_s_True = Check_human_factors(dic_entry_record, dic_rational_record, FLAG_TASK='SENT_PRED', current_word=current_word, current_word_list=word_list)
                            # T_sentence += T_temp
                            # KS_sentence += KS_temp
                            # if BOOL_RATIONAL_P_pred_s_True:
                                # check sentence pred
                            
                            dic_entry_record = Update_dic_entry_record(dic_entry_record, T_total_temp=T_key, T_key_total_temp=T_key, KS_total_temp=1, KS_select_total_temp=1) 
                            T_sentence += T_key   
                            KS_sentence += 1 
                            if BOOL_PRINT:
                                print(f"->>> KS_total={dic_entry_record['KS_total']}. A rational action, select sentence pred")
                            
                            current_word_index = len(word_list)-1
                            if BOOL_PRINT:
                                print("Sentence finished")
                                print("------------------------------------------------------------")
                            BOOL_SENTENCE_UNFINISH = False    
                        else:
                            # didn't find satisfying sentence, entry next word 
                            if BOOL_PRINT:
                                print("Not obtain sentence pred, entry next word")
                            dic_entry_record, dic_rational_record, T_temp, KS_temp, next_word, BOOL_SENTENCE_UNFINISH = Enter_next_word(dic_entry_record, dic_rational_record, current_word, word_list)  
                            T_sentence += T_temp
                            KS_sentence += KS_temp
                    else:
                        dic_rational_record['BOOL_RECORD_S_ON'] = True
                        dic_rational_record['BOOL_RECORD_W_ON'] = False
                        dic_rational_record['BOOL_RECORD_L_ON'] = False
                        dic_rational_record['BOOL_RECORD_ON'] = False
                        if BOOL_PRINT:
                            print("An irrational action: sentence level. not check sentence pred, type next word")
                            # print(f"BOOL_RECORD_S_ON: {dic_rational_record['BOOL_RECORD_S_ON']} - {dic_entry_record['KS_extra_s']}, BOOL_RECORD_W_ON: {dic_rational_record['BOOL_RECORD_W_ON']} - {dic_entry_record['KS_extra_w']}, BOOL_RECORD_L_ON: {dic_rational_record['BOOL_RECORD_L_ON']} - {dic_entry_record['KS_extra_l']}")
                        
                        dic_entry_record, dic_rational_record, T_temp, KS_temp, next_word, BOOL_SENTENCE_UNFINISH = Enter_next_word(dic_entry_record, dic_rational_record, current_word, word_list)
                        dic_entry_record = Update_dic_entry_record(dic_entry_record, N_bounded_s_total_temp=1, T_bounded_s_total_temp=T_temp, KS_bounded_s_total_temp=KS_temp)
                        dic_rational_record['BOOL_RECORD_ON'] = True
                        dic_entry_record = Record_extra_actions(dic_entry_record, dic_rational_record, T_extra_temp=T_temp-T_key, KS_extra_temp=KS_temp-1) # there is a choose action that needs to be subtracted
                        T_sentence += T_temp
                        KS_sentence += KS_temp
                        # Update the time saving
                        dic_entry_record = Record_time_saving_from_reactions(dic_entry_record, dic_rational_record, T_react_saving_temp=T_react_s)
                        dic_rational_record['BOOL_RECORD_S_ON'] = False
                        dic_rational_record['BOOL_RECORD_ON'] = False
                        
                        # if BOOL_PRINT:
                            # print(f"BOOL_RECORD_S_ON: {dic_rational_record['BOOL_RECORD_S_ON']} - {dic_entry_record['KS_extra_s']}, BOOL_RECORD_W_ON: {dic_rational_record['BOOL_RECORD_W_ON']} - {dic_entry_record['KS_extra_w']}, BOOL_RECORD_L_ON: {dic_rational_record['BOOL_RECORD_L_ON']} - {dic_entry_record['KS_extra_l']}")
                            # print("Sentence finished - 2")
                            # print("------------------------------------------------------------")
                        current_word_index = Find_current_word_index(current_word, word_list) 
                        if next_word == '':
                            BOOL_SENTENCE_UNFINISH = False
                else:
                    # do not check sentence pred, just entry next word
                    if BOOL_PRINT:
                        print("Not reach sentence type-then-look, type next word")
                    dic_entry_record, dic_rational_record, T_temp, KS_temp, next_word, BOOL_SENTENCE_UNFINISH = Enter_next_word(dic_entry_record, dic_rational_record, current_word, word_list)
                    T_sentence += T_temp
                    KS_sentence += KS_temp
    
    if next_word == '':
        BOOL_SENTENCE_UNFINISH = False

    return dic_entry_record, dic_rational_record, T_sentence, KS_sentence, next_word, BOOL_SENTENCE_UNFINISH 

### Enter next word

In [ ]:
def Find_current_word_index(current_word, word_list):
    current_word_index = 0
    i = 0
    for i, word in enumerate(word_list):
        # assume all words are different in a sentence
        if current_word == word:
            current_word_index = i
            break
    return current_word_index

In [ ]:
def Enter_next_word(dic_entry_record, dic_rational_record, current_word, word_list):

    
    current_word_index = Find_current_word_index(current_word, word_list)

    T_next_word = 0
    KS_next_word = 0
    # look at word pred
    BOOL_SENTENCE_UNFINISH = True

    dic_entry_record, dic_rational_record, T_temp, KS_temp, BOOL_RATIONAL_check_word_pred = Check_human_factors(dic_entry_record, dic_rational_record, FLAG_TASK='WORD_PRED', letter_index=0, current_word=current_word)
    T_next_word += T_temp
    KS_next_word += KS_temp
    if BOOL_RATIONAL_check_word_pred:
        if BOOL_PRINT:
            print("A rational action, check word pred")
        dic_entry_record = Update_dic_entry_record(dic_entry_record, T_total_temp=T_react_w, T_react_w_total_temp=T_react_w)
        T_next_word += T_react_w

        if random.random() < P_pred_w_n:
            if BOOL_PRINT:
                print("Obtained next word pred")
            # dic_entry_record, dic_rational_record, T_temp, KS_temp, BOOL_RATIONAL_P_pred_w_n = Check_human_factors(dic_entry_record, dic_rational_record, FLAG_TASK='LETTER', current_letter=' ')
            # T_next_word += T_temp
            # KS_next_word += KS_temp
            # if BOOL_RATIONAL_P_pred_w_n:
                # next word is obtained, click to select

            dic_entry_record = Update_dic_entry_record(dic_entry_record, T_total_temp=T_key, T_key_total_temp=T_key, KS_total_temp=1, KS_select_total_temp=1)
            T_next_word += T_key
            KS_next_word += 1
            current_word_index += 1
            current_word = word_list[current_word_index]
            if BOOL_PRINT:
                print(f"-> KS_total={dic_entry_record['KS_total']}. Select next word: {current_word}. A rational action")
            dic_entry_record, dic_rational_record, T_temp, KS_temp, current_word, BOOL_SENTENCE_UNFINISH = Check_sentence_finish(dic_entry_record, dic_rational_record, current_word, word_list)
            T_next_word += T_temp
            KS_next_word += KS_temp 
            
            # else:
                
        else:
            # next word is not obtained, entry next "current word"
            if BOOL_PRINT:
                print("Not obtained next word pred, type letters of the next word")
            current_word_index += 1
            current_word = word_list[current_word_index]
            dic_entry_record, dic_rational_record, next_word, T_temp, KS_temp, BOOL_WORD_SELECTED = Enter_current_word(dic_entry_record, dic_rational_record, word_list, current_word) 
            T_next_word += T_temp
            KS_next_word += KS_temp
            if current_word == word_list[-1]:
                BOOL_SENTENCE_UNFINISH = False
    else:
        if BOOL_PRINT:
            print("An irrational action: word level. not check word pred, type letters of the next word")
        if dic_rational_record['BOOL_RECORD_S_ON'] == False:
            dic_rational_record['BOOL_RECORD_W_ON'] = True
            dic_rational_record['BOOL_RECORD_L_ON'] = False
            dic_rational_record['BOOL_RECORD_ON'] = False
        # print(f"BOOL_RECORD_S_ON: {dic_rational_record['BOOL_RECORD_S_ON']} - {dic_entry_record['KS_extra_s']}, BOOL_RECORD_W_ON: {dic_rational_record['BOOL_RECORD_W_ON']} - {dic_entry_record['KS_extra_w']}, BOOL_RECORD_L_ON: {dic_rational_record['BOOL_RECORD_L_ON']} - {dic_entry_record['KS_extra_l']}")

        current_word_index += 1
        current_word = word_list[current_word_index]
        dic_entry_record, dic_rational_record, next_word, T_temp, KS_temp, BOOL_WORD_SELECTED = Enter_current_word(dic_entry_record, dic_rational_record, word_list, current_word) 
        dic_entry_record = Update_dic_entry_record(dic_entry_record, N_bounded_w_total_temp=1, T_bounded_w_total_temp=T_temp, KS_bounded_w_total_temp=KS_temp)
        if dic_rational_record['BOOL_RECORD_S_ON'] == False:
            dic_rational_record['BOOL_RECORD_ON'] = True
        dic_entry_record = Record_extra_actions(dic_entry_record, dic_rational_record, T_extra_temp=T_temp-T_key, KS_extra_temp=KS_temp-1) # there is a choose pred action that needs to be subtracted
        T_next_word += T_temp
        KS_next_word += KS_temp
        # Update the time saving
        dic_entry_record = Record_time_saving_from_reactions(dic_entry_record, dic_rational_record, T_react_saving_temp=T_react_w)
        dic_rational_record['BOOL_RECORD_W_ON'] = False
        if dic_rational_record['BOOL_RECORD_S_ON'] == False:
            dic_rational_record['BOOL_RECORD_ON'] = False
            
    return dic_entry_record, dic_rational_record, T_next_word, KS_next_word, current_word, BOOL_SENTENCE_UNFINISH

### Type one sentence

In [ ]:
def Type_one_sentence(sentence, L_min_w, k_look_w, p_max_w, L_min_s, k_look_s, p_max_s, R_rational=1, R_error=0, R_interrupt=0, T_event=5):

    T_total = 0
    T_key_total = 0
    T_react_w_total = 0
    T_react_s_total = 0
    T_react_total = 0
    T_bounded_l_total = 0
    T_bounded_w_total = 0
    T_bounded_s_total = 0
    T_error_total = 0
    T_interrupt_total = 0

    T_extra_s = 0
    T_extra_w = 0
    T_extra_l = 0
    T_extra_total = 0
    T_react_saving_s = 0
    T_react_saving_w = 0
    T_react_saving_l = 0
    T_react_saving_total = 0
    T_net_extra = 0

    N_bounded_l_total = 0
    N_bounded_w_total = 0
    N_bounded_s_total = 0
    N_bounded_total = 0
    T_event_total = 0
    
    KS_total = 0
    KS_type_total = 0
    KS_select_total = 0
    KS_bounded_l_total = 0
    KS_bounded_w_total = 0
    KS_bounded_s_total = 0
    KS_error_total = 0
    KS_interrupt_total = 0
    KS_extra_s = 0
    KS_extra_w = 0
    KS_extra_l = 0
    KS_extra_total = 0

    dic_entry_record = {
        "L_min_w": float(L_min_w),
        "k_look_w": float(k_look_w),
        "p_max_w": float(p_max_w),
        "L_min_s": float(L_min_s),
        "k_look_s": float(k_look_s),
        "p_max_s": float(p_max_s),
        
        'R_rational': float(R_rational),
        'R_error': float(R_error),
        'R_interrupt': float(R_interrupt),
        'T_event': float(T_event),

        "T_total": float(T_total),
        "T_key_total": float(T_key_total),
        "T_react_w_total": float(T_react_w_total),
        "T_react_s_total": float(T_react_s_total),
        "T_react_total": float(T_react_total),
        "T_bounded_l_total": float(T_bounded_l_total),
        "T_bounded_w_total": float(T_bounded_w_total),
        "T_bounded_s_total": float(T_bounded_s_total),
        "T_error_total": float(T_error_total),
        "T_interrupt_total": float(T_interrupt_total),
        "T_extra_s": float(T_extra_s),
        "T_extra_w": float(T_extra_w),
        "T_extra_l": float(T_extra_l),
        "T_extra_total": float(T_extra_total),
        "T_react_saving_s": float(T_react_saving_s),
        "T_react_saving_w": float(T_react_saving_w),
        "T_react_saving_l": float(T_react_saving_l),
        "T_react_saving_total": float(T_react_saving_total),
        "T_net_extra": float(T_net_extra),


        "N_bounded_l_total": int(N_bounded_l_total),
        "N_bounded_w_total": int(N_bounded_w_total),
        "N_bounded_s_total": int(N_bounded_s_total),
        "N_bounded_total": int(N_bounded_total),
        "T_event_total": float(T_event_total),

        "KS_total": int(KS_total),
        "KS_type_total": int(KS_type_total),
        "KS_select_total": int(KS_select_total),
        "KS_bounded_l_total": int(KS_bounded_l_total),
        "KS_bounded_w_total": int(KS_bounded_w_total),
        "KS_bounded_s_total": int(KS_bounded_s_total),
        "KS_error_total": int(KS_error_total),
        "KS_interrupt_total": int(KS_interrupt_total),
        "KS_extra_s": int(KS_extra_s),
        "KS_extra_w": int(KS_extra_w),
        "KS_extra_l": int(KS_extra_l),
        "KS_extra_total": int(KS_extra_total)
    }


    dic_rational_record = {
        "BOOL_RATIONAL_S": True,
        "BOOL_RATIONAL_W": True,
        "BOOL_RATIONAL_L": True,
        "BOOL_RECORD_NO": False,
        "BOOL_RECORD_S_ON": False,
        "BOOL_RECORD_W_ON": False,
        "BOOL_RECORD_L_ON": False
    }
    
    BOOL_SENTENCE_UNFINISH = True

    # sentence = "Do you know where the engineering department is?" #
    # sentence = "aaaaa bbbbb ccccc ddddd eeeee fffff ggggg hhhhh iiiii jjjjj kkkkk lllll mmmmm nnnnn ooooo ppppp qqqqq."
    # sentence = "How are you?"
    sentence = sentence.strip()
    sentence = re.sub(r'[^\w\s]', '', sentence)
    sentence = sentence.lower()
    # sentence = sentence + '.'
    word_list = sentence.split(' ')
    current_word = word_list[0]

    # entry current word
    dic_entry_record, dic_rational_record, next_word, T_temp, KS_temp, BOOL_WORD_SELECTED = Enter_current_word(dic_entry_record, dic_rational_record, word_list, current_word)
    T_total += T_temp
    KS_total += KS_temp
    if next_word == '':
        BOOL_SENTENCE_UNFINISH = False
        if BOOL_WORD_SELECTED == False:
            if BOOL_PRINT:
                print("Last word typed, remove the space after the final word")
            T_total -= T_key
            KS_total -= 1
            # Special dic_entry_record
            dic_entry_record['T_total'] -= T_key
            dic_entry_record['T_key_total'] -= T_key
            dic_entry_record['KS_total'] -= 1
            dic_entry_record['KS_type_total'] -= 1
    # else:
    #     current_word = next_word
        

    while BOOL_SENTENCE_UNFINISH:
    # check if the sentence is finished and execute follow up process
        dic_entry_record, dic_rational_record, T_temp, KS_temp, current_word, BOOL_SENTENCE_UNFINISH = Check_sentence_finish(dic_entry_record, dic_rational_record, current_word, word_list)
        T_total += T_temp
        KS_total += KS_temp
    
    # Check interruption
    R_interrupt = dic_entry_record['R_interrupt']
    T_event = dic_entry_record['T_event']
    if random.random() < R_interrupt:
        # T_event = 0 
        # T_event = 60*random.random()
        T_total += T_interrupt(T_event)
        if BOOL_PRINT:
            print(f"Interruption event: {T_event} s. Interruption time {T_interrupt(T_event)}")
        dic_entry_record = Update_dic_entry_record(dic_entry_record, T_event_total_temp=T_event)
        T_total_entry_rate = T_total - T_interrupt(T_event) + T_event
    else:
        T_total_entry_rate = T_total
    
    if BOOL_PRINT:
        print(f"T_total_entry_rate = {T_total_entry_rate}")
        print(f"T_total = {T_total}")
    
    # Calculate the overall impact of bounded rationality
    dic_entry_record['T_net_extra'] = dic_entry_record['T_extra_total'] - dic_entry_record['T_react_saving_total']

    if BOOL_PRINT:
        print(f"Time: {T_total}")
        print(f"Keystroke: {KS_total}")
        print(f"Extra actions count: {dic_entry_record['KS_extra_total']}")
        print(f"Extra action time: {dic_entry_record['T_extra_total']}")
        print(f"Saved time from reactions: {dic_entry_record['T_react_saving_total']}")
        print(f"Net extra time: {dic_entry_record['T_net_extra']}")
        print("^^^^^^^^^^^^^^^^^^^^^^^^^")

    return dic_entry_record, T_total, KS_total, T_total_entry_rate


#### Single sentence test

In [ ]:
# sentence = "Do you know where the engineering department is?"
# Only test word entry strategy

T_key =  0.26 # 0.26 # 0.6
T_react_w = 0.45 # 0.45 # 1.20
T_react_s = 1.87 # T_react_w * L_s_ave = 1.20*4.23 = 5.08

    
L_min_w = 3
k_look_w = 2
p_max_w = 5

# L_min_w_n = 1
# k_look_w_n = 1
# p_max_w_n = 9

L_min_s = 3
k_look_s = 1
p_max_s = 6

P_pred_w_c = 0.712
P_pred_w_n = 0.575
P_pred_s = 0.438

# def P_pred_s(current_word_index, L_s): # the probability of the system have correct sentence predictions
#     a=1
#     b=L_s
#     x=current_word_index
#     c=0
#     return a*math.log(x+1,b)+c

R_rational = 1
R_error = 0
R_interrupt = 0
T_event = 5

# R_interrupt = 0.0
# T_event = 60*random.random()
# def T_interrupt(T_event): # the probability of the system have correct sentence predictions
#     a=0.189
#     c=1.03
#     return a*math.log(T_event)+c

BOOL_PRINT = True

sentence = "hello where is engineering building" # "hello where is" hello where is engineering building
dic_entry_record = Type_one_sentence(sentence, L_min_w=L_min_w, k_look_w=k_look_w, p_max_w=p_max_w, L_min_s=L_min_s, k_look_s=k_look_s, p_max_s=p_max_s, R_rational=R_rational, R_error=R_error, R_interrupt=R_interrupt, T_event=T_event)
print(dic_entry_record)

# MobileHCI'23 Experiment

## 1. Dataset
Dataset is the same as the one we use for real user data collection to minimize the noice. 

In [ ]:
with open('raw_data.txt') as file:
    lines = file.readlines()

average_sentence_length = 0
for line in lines:
    line = re.sub(r'[^\w\s]', '', line)
    word_list = line.split(' ')
    average_sentence_length += len(word_list)

average_sentence_length = average_sentence_length/len(lines)
print(f"Average sentence length: {average_sentence_length}")

## 2. Experiment 1: KLM
AAC surrogate user VS. Able-bodied surrogate user

AAC:
1. Word Pred - NER
2. Word + Sen Pred - KSR
3. Word + Sen Pred - NER
4. Word + Sen Pred - NER_STD

Able-bodied:
1. Word + Sen Pred - NER
2. Word + Sen Pred - NER_STD

### 2.1. Load Envelope Analysis Simulator

In [ ]:
def Run(L_min_w=100.0, k_look_w=0.0, p_max_w=0.0, L_min_s=100.0, k_look_s=0.0, p_max_s=0.0, R_rational=1.0, R_error=0.0, R_interrupt=0.0, T_event=5.0):
    with open(f'raw_data.txt') as file:
        samples = file.readlines()
    # samples = ["Oh so what did dad have to say to you?"]
    data_dic_list = []

    for i, s in enumerate(samples):
        # print(f"In sample {i}/{len(samples)}: {s}")
        sample = s.strip()
        sample = re.sub(r'[^\w\s]', '', sample)
        word_list = sample.split(' ')
        dic_entry_record, T_total, KS_total, T_total_entry_rate = Type_one_sentence(sample, L_min_w, k_look_w, p_max_w, L_min_s, k_look_s, p_max_s, R_rational, R_error, R_interrupt, T_event) # Take_average_of_one_sentence_result(1, sample)
        data_dic = {
            # settings
            "L_min_w": float(L_min_w),
            "k_look_w": float(k_look_w),
            "p_max_w": float(p_max_w),
            "L_min_s": float(L_min_s),
            "k_look_s": float(k_look_s),
            "p_max_s": float(p_max_s),
            "P_pred_w_c": float(P_pred_w_c),
            "P_pred_w_n": float(P_pred_w_n),
            "T_key": float(T_key),
            "T_react_w": float(T_react_w),
            "T_react_s": float(T_react_s),
            "R_rational": float(R_rational), # bounded rationality index
            "R_error": float(R_error), # human error rate
            "R_interrupt": float(R_interrupt), # interruption rate
            "T_event": float(T_event), # interruption time

            # results
            "sentence": sample,
            "sentence_length": int(len(sample)),
            "sentence_length_in_words": int(len(word_list)),

            
            "N_bounded_l_total": dic_entry_record['N_bounded_l_total'],
            "N_bounded_w_total": dic_entry_record['N_bounded_w_total'], 
            "N_bounded_s_total": dic_entry_record['N_bounded_s_total'], 
            "N_bounded_total": dic_entry_record['N_bounded_total'],


            # T: 
            "T_total": dic_entry_record['T_total'],
            "T_key_total": dic_entry_record['T_key_total'],
            "T_react_w_total": dic_entry_record['T_react_w_total'],
            "T_react_s_total": dic_entry_record['T_react_s_total'],
            "T_react_total": dic_entry_record['T_react_total'],
            # bounded rationality
            "T_extra_total": dic_entry_record['T_extra_total'],
            "T_extra_s": dic_entry_record['T_extra_s'],
            "T_extra_w": dic_entry_record['T_extra_w'],
            "T_extra_l": dic_entry_record['T_extra_l'],
            "T_net_extra": dic_entry_record['T_net_extra'],
            # human error
            "T_error_total": dic_entry_record['T_error_total'],       
            # interruption
            "T_interrupt_total": dic_entry_record['T_interrupt_total'],
            "T_event_total": dic_entry_record['T_event_total'],

            # KS: 
            "KS_total": dic_entry_record['KS_total'],
            "KS_type_total": dic_entry_record['KS_type_total'],
            "KS_select_total": dic_entry_record['KS_select_total'],
            # bounded rationality
            "KS_extra_total": dic_entry_record['KS_extra_total'],
            "KS_extra_s": dic_entry_record['KS_extra_s'],
            "KS_extra_w": dic_entry_record['KS_extra_w'],
            "KS_extra_l": dic_entry_record['KS_extra_l'],
            # human error
            "KS_error_total": dic_entry_record['KS_error_total'],
            # interruption no KS

            # basic statistics
            "entry_rate": float(len(sample)/T_total_entry_rate)*12.0, # wpm
            "net_entry_rate": float((len(sample)/T_total)*12.0 - (12.0/T_key)), # wpm
            "KS_saving": float(float(len(sample)) - KS_total),
            "KS_saving_rate": float(float(len(sample)) - KS_total)/float(len(sample)), # %

            # Human performance analyses
            "effective_typing_rate_index": float(  ((dic_entry_record['KS_total']-dic_entry_record['KS_error_total'])/dic_entry_record['T_key_total']) * T_key ),
            "error_index": float(dic_entry_record['KS_error_total']/(dic_entry_record['KS_total']-dic_entry_record['KS_extra_total'])),
            "rational_index": float(1- ( dic_entry_record['KS_extra_total']/(dic_entry_record['KS_total']-dic_entry_record['KS_error_total']) ) ),
            "intuitiveness_index_KS": float(1- ( (dic_entry_record['KS_error_total']+dic_entry_record['KS_extra_total'])/dic_entry_record['KS_total'] ) ),
            "intuitiveness_index_T": float(1- ( (dic_entry_record['T_error_total']+dic_entry_record['T_net_extra'])/dic_entry_record['T_total'] ) )

        }

        data_dic_list.append(data_dic)

    # data_dic_list.sort(key=sort_by_sentence_length)

    # Collect data
    df_data_dic_list = pd.DataFrame(data=data_dic_list)

    return df_data_dic_list

### 2.2. Set User and System Parameters

#### 2.2.1. Sharable Parameters

In [ ]:
R_rational = 1
R_error = 0
R_interrupt = 0

T_event = 5

def T_interrupt(T_event): # the probability of the system have correct sentence predictions
    a=0.189
    c=1.03
    return a*math.log(T_event)+c

# Prediction settings
P_pred_w_c = 0.712 # 0.6 # KWickChat
P_pred_w_n = 0.575 # 0.5 # KWickChat
P_pred_s = 0.438

#### 2.2.2. AAC User Parameters

In [ ]:
T_key =  0.6 # 0.26 # 0.6
T_react_w = 1.20 # 0.45 # 1.20
T_react_s = 6.16 # 1.90 # T_react_w * L_s_ave = 1.20*5.13 = 6.16

#### 2.2.3. Able-bodied User Parameters

In [ ]:
T_key =  0.26 # 0.6
T_react_w = 0.45 # 1.20
T_react_s = 2.31 # T_react_w * L_s_ave = 0.45*5.13 = 2.31

#### 2.2.4. Strategy Parameters: Word Only

In [ ]:
# Text entering strategy
L_min_w = 2 # 2-10
k_look_w = 1 # 0-5
p_max_w = 2
L_min_s = 100
k_look_s = 0
p_max_s = 0

#### 2.2.5. Strategy Parameters: Sen Only

In [ ]:
# Text entering strategy
L_min_w = 100 
k_look_w = 0 
p_max_w = 0
L_min_s = 2 # 2-10
k_look_s = 1 # 0-5 
p_max_s = 0

#### 2.2.6. Strategy Parameters: Word + Sen

In [ ]:
# Text entering strategy
L_min_w = 4 # 2-10
k_look_w = 1 
p_max_w = 6
L_min_s = 5 # 2-10
k_look_s = 1
p_max_s = 6

### 2.3. AAC Users

#### 2.3.1. Word Pred Only

##### 2.3.1.1. L vs k

In [ ]:
def AAC_Word_L_k():
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(0,5.1,0.1):
            data_frame = Run(L_min_w=i, k_look_w=j, p_max_w=p_max_w, L_min_s=L_min_s, k_look_s=k_look_s, p_max_s=p_max_s, R_rational=R_rational, R_error=R_error, R_interrupt=R_interrupt, T_event=T_event)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'231_AAC_Word_L_k.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results

In [ ]:
BOOL_PRINT = False
aac_word_L_k_results = AAC_Word_L_k()

In [ ]:
# Read from file
readname = f'231_AAC_Word_L_k.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
aac_word_L_k_csv = pd.read_csv(fullname)

In [ ]:
# evaluate L_min_w and k_look_w
aac_word_L_k_data = aac_word_L_k_csv[['net_entry_rate', 'L_min_w', 'k_look_w']]
vis_net_entry_rate_mean = aac_word_L_k_data.groupby(['L_min_w', 'k_look_w']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

# vis_net_entry_rate_std = con_ab_net_entry_rate_L_min_w_k_look_w.groupby(['L_min_w', 'k_look_w']).std()
# vis_net_entry_rate_std = vis_net_entry_rate_std.reset_index()


fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('k_look_w', 'L_min_w', 'net_entry_rate')
# pivot_table_std = vis_net_entry_rate_std.pivot('k_look_w', 'L_min_w', 'net_entry_rate')
# print(pivot_table.shape)
# print(pivot_table_std.shape)


plt.xlabel('L_min_w', size=25)
plt.ylabel('k_look_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)
# cbar.set_ticks(np.arange(pivot_table.values.min,pivot_table.values.max,8))
# cbar.set_ticks(np.arange(0,5,8))
# cbar.set_ticklabels(np.linspace(0,5,8, dtype=np.float64))

ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(0, 5, 6, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'231_AAC_Word_L_k.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)

##### 2.3.1.2. L vs p

In [ ]:
# Text entering strategy
L_min_w = 2 # 2-10
k_look_w = 1 # 0-5
p_max_w = 2 # 2-10
L_min_s = 100
k_look_s = 0
p_max_s = 0

In [ ]:
def AAC_Word_L_p():
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(2,10.1,0.1):
            data_frame = Run(L_min_w=i, k_look_w=k_look_w, p_max_w=j, L_min_s=L_min_s, k_look_s=k_look_s, p_max_s=p_max_s, R_rational=R_rational, R_error=R_error, R_interrupt=R_interrupt, T_event=T_event)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'2312_AAC_Word_L_p.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results

In [ ]:
BOOL_PRINT = False
aac_word_L_p_results = AAC_Word_L_p()

In [ ]:
# Read from file
readname = f'2312_AAC_Word_L_p.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
aac_word_L_p_csv = pd.read_csv(fullname)

In [ ]:
# evaluate L_min_w and p_max_w
aac_word_L_p_data = aac_word_L_p_csv[['net_entry_rate', 'L_min_w', 'p_max_w']]
vis_net_entry_rate_mean = aac_word_L_p_data.groupby(['L_min_w', 'p_max_w']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

# vis_net_entry_rate_std = con_ab_net_entry_rate_L_min_w_k_look_w.groupby(['L_min_w', 'k_look_w']).std()
# vis_net_entry_rate_std = vis_net_entry_rate_std.reset_index()


fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('p_max_w', 'L_min_w', 'net_entry_rate')
# pivot_table_std = vis_net_entry_rate_std.pivot('k_look_w', 'L_min_w', 'net_entry_rate')
# print(pivot_table.shape)
# print(pivot_table_std.shape)


plt.xlabel('L_min_w', size=25)
plt.ylabel('p_max_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)
# cbar.set_ticks(np.arange(pivot_table.values.min,pivot_table.values.max,8))
# cbar.set_ticks(np.arange(0,5,8))
# cbar.set_ticklabels(np.linspace(0,5,8, dtype=np.float64))

ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'2312_AAC_Word_L_p.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)

##### 2.3.1.3. p vs k

In [ ]:
# Text entering strategy
L_min_w = 4 # 2-10
k_look_w = 1 # 0-5
p_max_w = 2 # 2-10
L_min_s = 100
k_look_s = 0
p_max_s = 0

In [ ]:
def AAC_Word_p_k():
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(0,5.1,0.1):
            data_frame = Run(L_min_w=L_min_w, k_look_w=j, p_max_w=i, L_min_s=L_min_s, k_look_s=k_look_s, p_max_s=p_max_s, R_rational=R_rational, R_error=R_error, R_interrupt=R_interrupt, T_event=T_event)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'2313_AAC_Word_p_k.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results

In [ ]:
BOOL_PRINT = False
aac_word_p_k_results = AAC_Word_p_k()

In [ ]:
# Read from file
readname = f'2313_AAC_Word_p_k.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
aac_word_p_k_csv = pd.read_csv(fullname)

In [ ]:
# evaluate p_max_w and k_look_w
aac_word_p_k_data = aac_word_p_k_csv[['net_entry_rate', 'p_max_w', 'k_look_w']]
vis_net_entry_rate_mean = aac_word_p_k_data.groupby(['p_max_w', 'k_look_w']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

# vis_net_entry_rate_std = con_ab_net_entry_rate_L_min_w_k_look_w.groupby(['L_min_w', 'k_look_w']).std()
# vis_net_entry_rate_std = vis_net_entry_rate_std.reset_index()


fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('k_look_w', 'p_max_w', 'net_entry_rate')
# pivot_table_std = vis_net_entry_rate_std.pivot('k_look_w', 'L_min_w', 'net_entry_rate')
# print(pivot_table.shape)
# print(pivot_table_std.shape)


plt.xlabel('p_max_w', size=25)
plt.ylabel('k_look_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)
# cbar.set_ticks(np.arange(pivot_table.values.min,pivot_table.values.max,8))
# cbar.set_ticks(np.arange(0,5,8))
# cbar.set_ticklabels(np.linspace(0,5,8, dtype=np.float64))

ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(0, 5, 6, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'2313_AAC_Word_p_k.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)

#### 2.3.2. Word + Sen Pred


In [ ]:
# Text entering strategy
L_min_w = 4 # 2-10
k_look_w = 0 
p_max_w = 2
L_min_s = 5 # 2-10
k_look_s = 0
p_max_s = 2

In [ ]:
def AAC_Word_Sen_L_L():
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(2,10.1,0.1):
            data_frame = Run(L_min_w=i, k_look_w=k_look_w, p_max_w=p_max_w, L_min_s=j, k_look_s=k_look_s, p_max_s=p_max_s)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'232_AAC_Word_Sen_L_L.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results


In [ ]:
BOOL_PRINT = False
aac_word_sen_L_L_results = AAC_Word_Sen_L_L()

In [ ]:
# Read from file
readname = f'232_AAC_Word_Sen_L_L.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
aac_word_sen_L_L_csv = pd.read_csv(fullname)

In [ ]:
# evaluate L_min_w and L_min_s to NER
aac_word_sen_L_L_data = aac_word_sen_L_L_csv[['net_entry_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = aac_word_sen_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'net_entry_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)


ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'232_AAC_Word_Sen_L_L_NER.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)




In [ ]:
# evaluate L_min_w and L_min_s to NER
aac_word_sen_L_L_data = aac_word_sen_L_L_csv[['net_entry_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_std = aac_word_sen_L_L_data.groupby(['L_min_w', 'L_min_s']).std()
vis_net_entry_rate_std = vis_net_entry_rate_std.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_std.pivot('L_min_w', 'L_min_s', 'net_entry_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=22)
print(pivot_table.values.max)


ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'232_AAC_Word_Sen_L_L_NER_STD.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


In [ ]:
# evaluate L_min_w and L_min_s to KSR
aac_word_sen_L_L_data = aac_word_sen_L_L_csv[['KS_saving_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = aac_word_sen_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'KS_saving_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=20)
# print(pivot_table.values.max)
cbar.ax.set_yticklabels(["{:.1%}".format(i) for i in cbar.get_ticks()])

ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'232_AAC_Word_Sen_L_L_KSR.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


#### 2.3.3. Sen Pred Only 

In [ ]:
# Text entering strategy
L_min_w = 100 
k_look_w = 0 
p_max_w = 0
L_min_s = 2 # 2-10
k_look_s = 1 # 0-5 
p_max_s = 2

##### 2.3.3.1. L vs k

In [ ]:
def AAC_Sen_L_k():
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(0,5.1,0.1):
            data_frame = Run(L_min_w=L_min_w, k_look_w=k_look_w, p_max_w=p_max_w, L_min_s=i, k_look_s=j, p_max_s=p_max_s, R_rational=R_rational, R_error=R_error, R_interrupt=R_interrupt, T_event=T_event)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'2331_AAC_Sen_L_k.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results

In [ ]:
BOOL_PRINT = False
aac_sen_L_k_results = AAC_Sen_L_k()

In [ ]:
# Read from file
readname = f'2331_AAC_Sen_L_k.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
aac_sen_L_k_csv = pd.read_csv(fullname)

In [ ]:
# evaluate L_min_w and k_look_w
aac_sen_L_k_data = aac_sen_L_k_csv[['net_entry_rate', 'L_min_s', 'k_look_s']]
vis_net_entry_rate_mean = aac_sen_L_k_data.groupby(['L_min_s', 'k_look_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

# vis_net_entry_rate_std = con_ab_net_entry_rate_L_min_w_k_look_w.groupby(['L_min_w', 'k_look_w']).std()
# vis_net_entry_rate_std = vis_net_entry_rate_std.reset_index()


fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('k_look_s', 'L_min_s', 'net_entry_rate')
# pivot_table_std = vis_net_entry_rate_std.pivot('k_look_w', 'L_min_w', 'net_entry_rate')
# print(pivot_table.shape)
# print(pivot_table_std.shape)


plt.xlabel('L_min_s', size=25)
plt.ylabel('k_look_s', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)
# cbar.set_ticks(np.arange(pivot_table.values.min,pivot_table.values.max,8))
# cbar.set_ticks(np.arange(0,5,8))
# cbar.set_ticklabels(np.linspace(0,5,8, dtype=np.float64))

ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(0, 5, 6, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'2331_AAC_Sen_L_k.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)

##### 2.3.3.2. L vs p

In [ ]:
def AAC_Sen_L_p():
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(2,10.1,0.1):
            data_frame = Run(L_min_w=L_min_w, k_look_w=k_look_w, p_max_w=p_max_w, L_min_s=i, k_look_s=k_look_s, p_max_s=j, R_rational=R_rational, R_error=R_error, R_interrupt=R_interrupt, T_event=T_event)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'2332_AAC_Sen_L_p.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results

In [ ]:
BOOL_PRINT = False
aac_sen_L_p_results = AAC_Sen_L_p()

In [ ]:
# Read from file
readname = f'2332_AAC_Sen_L_p.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
aac_sen_L_p_csv = pd.read_csv(fullname)

In [ ]:
# evaluate L_min_w and p_max_w
aac_sen_L_p_data = aac_sen_L_p_csv[['net_entry_rate', 'L_min_s', 'p_max_s']]
vis_net_entry_rate_mean = aac_sen_L_p_data.groupby(['L_min_s', 'p_max_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

# vis_net_entry_rate_std = con_ab_net_entry_rate_L_min_w_k_look_w.groupby(['L_min_w', 'k_look_w']).std()
# vis_net_entry_rate_std = vis_net_entry_rate_std.reset_index()


fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('p_max_s', 'L_min_s', 'net_entry_rate')
# pivot_table_std = vis_net_entry_rate_std.pivot('k_look_w', 'L_min_w', 'net_entry_rate')
# print(pivot_table.shape)
# print(pivot_table_std.shape)


plt.xlabel('L_min_s', size=25)
plt.ylabel('p_max_s', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)
# cbar.set_ticks(np.arange(pivot_table.values.min,pivot_table.values.max,8))
# cbar.set_ticks(np.arange(0,5,8))
# cbar.set_ticklabels(np.linspace(0,5,8, dtype=np.float64))

ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'2332_AAC_Sen_L_p.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)

##### 2.3.3.3. p vs k

In [ ]:
# Text entering strategy
L_min_w = 100 
k_look_w = 0 
p_max_w = 0
L_min_s = 5 
k_look_s = 1 # 0-5
p_max_s = 6 # 2-10

In [ ]:
def AAC_Sen_p_k():
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(0,5.1,0.1):
            data_frame = Run(L_min_w=L_min_w, k_look_w=k_look_w, p_max_w=p_max_w, L_min_s=L_min_s, k_look_s=j, p_max_s=i, R_rational=R_rational, R_error=R_error, R_interrupt=R_interrupt, T_event=T_event)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'2333_AAC_Sen_p_k.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results

In [ ]:
BOOL_PRINT = False
aac_sen_p_k_results = AAC_Sen_p_k()

In [ ]:
# Read from file
readname = f'2333_AAC_Sen_p_k.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
aac_sen_p_k_csv = pd.read_csv(fullname)

In [ ]:
# evaluate p_max_w and k_look_w
aac_sen_p_k_data = aac_sen_p_k_csv[['net_entry_rate', 'p_max_s', 'k_look_s']]
vis_net_entry_rate_mean = aac_sen_p_k_data.groupby(['p_max_s', 'k_look_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

# vis_net_entry_rate_std = con_ab_net_entry_rate_L_min_w_k_look_w.groupby(['L_min_w', 'k_look_w']).std()
# vis_net_entry_rate_std = vis_net_entry_rate_std.reset_index()


fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('k_look_s', 'p_max_s', 'net_entry_rate')
# pivot_table_std = vis_net_entry_rate_std.pivot('k_look_w', 'L_min_w', 'net_entry_rate')
# print(pivot_table.shape)
# print(pivot_table_std.shape)


plt.xlabel('p_max_s', size=25)
plt.ylabel('k_look_s', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)
# cbar.set_ticks(np.arange(pivot_table.values.min,pivot_table.values.max,8))
# cbar.set_ticks(np.arange(0,5,8))
# cbar.set_ticklabels(np.linspace(0,5,8, dtype=np.float64))

ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(0, 5, 6, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'2333_AAC_Sen_p_k.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)

### 2.4. Able-bodied Users

#### 2.4.1. Word + Sen Pred


In [ ]:
def able_Word_Sen_L_L():
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(2,10.1,0.1):
            data_frame = Run(L_min_w=i, k_look_w=k_look_w, p_max_w=p_max_w, L_min_s=j, k_look_s=k_look_s, p_max_s=p_max_s)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'241_able_Word_Sen_L_L.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results


In [ ]:
BOOL_PRINT = False
able_word_sen_L_L_results = able_Word_Sen_L_L()

In [ ]:
# Read from file
readname = f'241_able_Word_Sen_L_L.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
able_word_sen_L_L_csv = pd.read_csv(fullname)

In [ ]:
# evaluate L_min_w and L_min_s to NER
able_word_sen_L_L_data = able_word_sen_L_L_csv[['net_entry_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = able_word_sen_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'net_entry_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)


ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'241_able_Word_Sen_L_L_NER.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)




In [ ]:
# evaluate L_min_w and L_min_s to NER_STD
able_word_sen_L_L_data = able_word_sen_L_L_csv[['net_entry_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_std = able_word_sen_L_L_data.groupby(['L_min_w', 'L_min_s']).std()
vis_net_entry_rate_std = vis_net_entry_rate_std.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_std.pivot('L_min_w', 'L_min_s', 'net_entry_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=22)
print(pivot_table.values.max)


ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'241_able_Word_Sen_L_L_NER_STD.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


In [ ]:
# evaluate L_min_w and L_min_s to KSR
aac_word_sen_L_L_data = aac_word_sen_L_L_csv[['KS_saving_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = aac_word_sen_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'KS_saving_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=20)
# print(pivot_table.values.max)
cbar.ax.set_yticklabels(["{:.1%}".format(i) for i in cbar.get_ticks()])

ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'232_AAC_Word_Sen_L_L_KSR.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


## 3. Experiment 2: Human Factors

### 3.1. User and System Parameters

In [ ]:
# Prediction settings
P_pred_w_c = 0.712 # 0.6 # KWickChat
P_pred_w_n = 0.575 # 0.5 # KWickChat
P_pred_s = 0.438

# AAC user
T_key =  0.6 # 0.26 # 0.6
T_react_w = 1.20 # 0.45 # 1.20
T_react_s = 6.16 # 1.90 # T_react_w * L_s_ave = 1.20*5.13 = 6.16

# Text entering strategy
L_min_w = 3 # 2-10
k_look_w = 0 
p_max_w = 2
L_min_s = 5 # 2-10
k_look_s = 0
p_max_s = 2

### 3.2. Bounded Rationality

In [ ]:
R_rational = 1
R_error = 0
R_interrupt = 0

T_event = 5

def T_interrupt(T_event): # the probability of the system have correct sentence predictions
    a=0.189
    c=1.03
    return a*math.log(T_event)+c

#### 3.2.1. AAC, Word + Sen Pred, Irrational


In [ ]:
def AAC_Word_Sen_B_L_L():
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(2,10.1,0.1):
            data_frame = Run(L_min_w=i, k_look_w=k_look_w, p_max_w=p_max_w, L_min_s=j, k_look_s=k_look_s, p_max_s=p_max_s, R_rational=R_rational, R_error=R_error, R_interrupt=R_interrupt, T_event=T_event)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'321_aac_Word_Sen_B_L_L.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results


In [ ]:
R_rational = 0.5

In [ ]:
BOOL_PRINT = False
aac_word_sen_b_L_L_results = AAC_Word_Sen_B_L_L()

In [ ]:
# Read from file
readname = f'321_aac_Word_Sen_B_L_L.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
aac_word_sen_b_L_L_csv = pd.read_csv(fullname)

In [ ]:
# evaluate L_min_w and L_min_s to NER
aac_word_sen_b_L_L_data = aac_word_sen_b_L_L_csv[['net_entry_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = aac_word_sen_b_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'net_entry_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)


ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'321_aac_Word_Sen_B_L_L_NER.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)




In [ ]:
# evaluate L_min_w and L_min_s to NER_STD
aac_word_sen_b_L_L_data = aac_word_sen_b_L_L_csv[['KS_extra_total', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = aac_word_sen_b_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'KS_extra_total')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)


ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'321_aac_Word_Sen_B_L_L_KS_extra.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)



In [ ]:
# evaluate L_min_w and L_min_s to KSR
aac_word_sen_b_L_L_data = aac_word_sen_b_L_L_csv[['KS_saving_rate', 'L_min_w', 'L_min_s']]
vis_KS_saving_rate_mean = aac_word_sen_b_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_KS_saving_rate_mean = vis_KS_saving_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_KS_saving_rate_mean.pivot('L_min_w', 'L_min_s', 'KS_saving_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=20)
# print(pivot_table.values.max)
cbar.ax.set_yticklabels(["{:.1%}".format(i) for i in cbar.get_ticks()])

ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'321_AAC_Word_Sen_b_L_L_KSR.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


### 3.3. Human Error

In [ ]:
R_rational = 1
R_error = 0
R_interrupt = 0

T_event = 5

def T_interrupt(T_event): # the probability of the system have correct sentence predictions
    a=0.189
    c=1.03
    return a*math.log(T_event)+c

#### 3.3.1. AAC, Word + Sen Pred, Error

In [ ]:
def AAC_Word_Sen_E_L_L():
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(2,10.1,0.1):
            data_frame = Run(L_min_w=i, k_look_w=k_look_w, p_max_w=p_max_w, L_min_s=j, k_look_s=k_look_s, p_max_s=p_max_s, R_rational=1.0, R_error=R_error, R_interrupt=0.0, T_event=5.0)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'331_aac_Word_Sen_E_L_L.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results


In [ ]:
R_error = 0.5

In [ ]:
BOOL_PRINT = False
aac_word_sen_e_L_L_results = AAC_Word_Sen_E_L_L()

In [ ]:
# Read from file
readname = f'331_aac_Word_Sen_E_L_L.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
aac_word_sen_e_L_L_csv = pd.read_csv(fullname)

In [ ]:
# evaluate L_min_w and L_min_s to NER
aac_word_sen_e_L_L_data = aac_word_sen_e_L_L_csv[['net_entry_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = aac_word_sen_e_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'net_entry_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)


ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'321_aac_Word_Sen_E_L_L_NER.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)




In [ ]:
# evaluate L_min_w and L_min_s to KSR
aac_word_sen_e_L_L_data = aac_word_sen_e_L_L_csv[['KS_error_total', 'L_min_w', 'L_min_s']]
vis_KS_saving_rate_mean = aac_word_sen_e_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_KS_saving_rate_mean = vis_KS_saving_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_KS_saving_rate_mean.pivot('L_min_w', 'L_min_s', 'KS_error_total')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=20)
# print(pivot_table.values.max)
# cbar.ax.set_yticklabels(["{:.1%}".format(i) for i in cbar.get_ticks()])

ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'331_AAC_Word_Sen_e_L_L_KSR.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


### 3.4. Interruption

#### 3.4.1. AAC, Word + Sen Pred, Interruption

In [ ]:
def AAC_Word_Sen_I_L_L():
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(2,10.1,0.1):
            data_frame = Run(L_min_w=i, k_look_w=k_look_w, p_max_w=p_max_w, L_min_s=j, k_look_s=k_look_s, p_max_s=p_max_s, R_rational=1.0, R_error=0.0, R_interrupt=R_interrupt, T_event=T_event)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'341_aac_Word_Sen_I_L_L.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results


In [ ]:
R_interrupt = 0.1
T_event = 5

In [ ]:
R_interrupt = 0.1
T_event = 50

In [ ]:
R_interrupt = 0.5
T_event = 5

In [ ]:
R_interrupt = 0.5
T_event = 50

In [ ]:
BOOL_PRINT = False
aac_word_sen_i_L_L_results = AAC_Word_Sen_I_L_L()

In [ ]:
# Read from file
readname = f'341_aac_Word_Sen_I_L_L.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
aac_word_sen_i_L_L_csv = pd.read_csv(fullname)

In [ ]:
# evaluate L_min_w and L_min_s to NER
aac_word_sen_i_L_L_data = aac_word_sen_i_L_L_csv[['net_entry_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = aac_word_sen_i_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'net_entry_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)


ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'341_aac_Word_Sen_I_L_L_NER.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)




### 3.5. KLM-BEI

#### 3.5.1. AAC, Word + Sen Pred, BEI

In [ ]:
def AAC_Word_Sen_BEI_B_E():
    exp_result_dic_list = []
    for i in np.arange(0,1.05,0.05):
        for j in np.arange(0,1.05,0.05):
            data_frame = Run(L_min_w=L_min_w, k_look_w=k_look_w, p_max_w=p_max_w, L_min_s=L_min_s, k_look_s=k_look_s, p_max_s=p_max_s, R_rational=i, R_error=j, R_interrupt=0.0, T_event=5.0)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'351_aac_Word_Sen_BEI_B_E.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results


In [ ]:
# Text entering strategy
L_min_w = 4 
k_look_w = 0
p_max_w = 2
L_min_s = 5 
k_look_s = 0
p_max_s = 2

In [ ]:
R_interrupt = 0.5

T_event = 5

def T_interrupt(T_event): # the probability of the system have correct sentence predictions
    a=0.189
    c=1.03
    return a*math.log(T_event)+c

In [ ]:
BOOL_PRINT = False
aac_word_sen_BEI_B_E_results = AAC_Word_Sen_BEI_B_E()

In [ ]:
# Read from file
readname = f'351_aac_Word_Sen_BEI_B_E.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
aac_word_sen_bei_B_E_csv = pd.read_csv(fullname)

In [ ]:
# evaluate R_rational and R_error to NER
aac_word_sen_bei_B_E_data = aac_word_sen_bei_B_E_csv[['net_entry_rate', 'R_rational', 'R_error']]
vis_net_entry_rate_mean = aac_word_sen_bei_B_E_data.groupby(['R_rational', 'R_error']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('R_rational', 'R_error', 'net_entry_rate')

plt.xlabel('R_error (%)', size=25)
plt.ylabel('R_rational (%)', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)


ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,5))
xticks = np.linspace(0, 100, 5, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,5))
yticks = np.linspace(0, 100, 5, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'351_aac_Word_Sen_BEI_B_E_NER.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)







In [ ]:
# evaluate R_rational and R_error to KSR
aac_word_sen_bei_B_E_data = aac_word_sen_bei_B_E_csv[['KS_saving_rate', 'R_rational', 'R_error']]
vis_net_entry_rate_mean = aac_word_sen_bei_B_E_data.groupby(['R_rational', 'R_error']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('R_rational', 'R_error', 'KS_saving_rate')

plt.xlabel('R_error (%)', size=25)
plt.ylabel('R_rational (%)', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)
cbar.ax.set_yticklabels(["{:.1%}".format(i) for i in cbar.get_ticks()])

ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,5))
xticks = np.linspace(0, 100, 5, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,5))
yticks = np.linspace(0, 100, 5, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'351_aac_Word_Sen_BEI_B_E_KSR.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)






#### 3.5.2. AAC, BEI, NER, KSR

In [ ]:
# AAC user 
T_key =  0.6 
T_react_w = 1.2 
T_react_s = 6.16 

R_rational = 0.8
R_error = 0.1
R_interrupt = 0.2
T_event = 5

# Prediction settings
P_pred_w_c = 0.712 # empirical data
P_pred_w_n = 0.575 # empirical data
P_pred_s = 0.438

# Text entering strategy
L_min_w = 4 # 2-10
k_look_w = 1
p_max_w = 6
L_min_s = 5 # 2-10
k_look_s = 1
p_max_s = 6


In [ ]:
def AAC_Word_Sen_BEI_L_L():
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(2,10.1,0.1):
            data_frame = Run(L_min_w=i, k_look_w=k_look_w, p_max_w=p_max_w, L_min_s=j, k_look_s=k_look_s, p_max_s=p_max_s, R_rational=R_rational, R_error=R_error, R_interrupt=R_interrupt, T_event=T_event)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'352_aac_Word_Sen_BEI_L_L.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results

In [ ]:
BOOL_PRINT = False
aac_word_sen_bei_L_L_results = AAC_Word_Sen_BEI_L_L()

In [ ]:
# Read from file
readname = f'352_aac_Word_Sen_BEI_L_L.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
aac_word_sen_bei_L_L_csv = pd.read_csv(fullname)

In [ ]:
# evaluate L_min_w and L_min_s to NER
aac_word_sen_bei_L_L_data = aac_word_sen_bei_L_L_csv[['net_entry_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = aac_word_sen_bei_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'net_entry_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)


ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'352_aac_Word_Sen_BEI_L_L_NER.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


In [ ]:
# evaluate L_min_w and L_min_s to NER
aac_word_sen_bei_L_L_data = aac_word_sen_bei_L_L_csv[['net_entry_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_std = aac_word_sen_bei_L_L_data.groupby(['L_min_w', 'L_min_s']).std()
vis_net_entry_rate_std = vis_net_entry_rate_std.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_std.pivot('L_min_w', 'L_min_s', 'net_entry_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)


ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'352_aac_Word_Sen_BEI_L_L_NER.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


In [ ]:
# evaluate L_min_w and L_min_s to KSR
aac_word_sen_bei_L_L_data = aac_word_sen_bei_L_L_csv[['KS_saving_rate', 'L_min_w', 'L_min_s']]
vis_ks_saving_rate_mean = aac_word_sen_bei_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_ks_saving_rate_mean = vis_ks_saving_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_ks_saving_rate_mean.pivot('L_min_w', 'L_min_s', 'KS_saving_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)
cbar.ax.set_yticklabels(["{:.1%}".format(i) for i in cbar.get_ticks()])

ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'352_aac_Word_Sen_BEI_L_L_KSR.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


In [ ]:
# evaluate L_min_w and L_min_s to KSR_STD
aac_word_sen_bei_L_L_data = aac_word_sen_bei_L_L_csv[['KS_saving_rate', 'L_min_w', 'L_min_s']]
vis_ks_saving_rate_mean = aac_word_sen_bei_L_L_data.groupby(['L_min_w', 'L_min_s']).std()
vis_ks_saving_rate_mean = vis_ks_saving_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_ks_saving_rate_mean.pivot('L_min_w', 'L_min_s', 'KS_saving_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)
# cbar.ax.set_yticklabels(["{:.1%}".format(i) for i in cbar.get_ticks()])

ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'352_aac_Word_Sen_BEI_L_L_KSR_STD.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


In [ ]:
# evaluate L_min_w and L_min_s to RI
aac_word_sen_bei_L_L_data = aac_word_sen_bei_L_L_csv[['rational_index', 'L_min_w', 'L_min_s']]
vis_ks_saving_rate_mean = aac_word_sen_bei_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_ks_saving_rate_mean = vis_ks_saving_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_ks_saving_rate_mean.pivot('L_min_w', 'L_min_s', 'rational_index')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)
cbar.ax.set_yticklabels(["{:.1%}".format(i) for i in cbar.get_ticks()])

ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'352_aac_Word_Sen_BEI_L_L_RI.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)

In [ ]:
# evaluate L_min_w and L_min_s to EI
aac_word_sen_bei_L_L_data = aac_word_sen_bei_L_L_csv[['error_index', 'L_min_w', 'L_min_s']]
vis_ks_saving_rate_mean = aac_word_sen_bei_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_ks_saving_rate_mean = vis_ks_saving_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_ks_saving_rate_mean.pivot('L_min_w', 'L_min_s', 'error_index')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)
cbar.ax.set_yticklabels(["{:.1%}".format(i) for i in cbar.get_ticks()])

ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'352_aac_Word_Sen_BEI_L_L_EI.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)

In [ ]:
# evaluate L_min_w and L_min_s to UFI_KS
aac_word_sen_bei_L_L_data = aac_word_sen_bei_L_L_csv[['intuitiveness_index_KS', 'L_min_w', 'L_min_s']]
vis_ks_saving_rate_mean = aac_word_sen_bei_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_ks_saving_rate_mean = vis_ks_saving_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_ks_saving_rate_mean.pivot('L_min_w', 'L_min_s', 'intuitiveness_index_KS')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)
cbar.ax.set_yticklabels(["{:.1%}".format(i) for i in cbar.get_ticks()])

ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'352_aac_Word_Sen_BEI_L_L_UFI_KS.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)

In [ ]:
# evaluate L_min_w and L_min_s to UFI_T
aac_word_sen_bei_L_L_data = aac_word_sen_bei_L_L_csv[['intuitiveness_index_T', 'L_min_w', 'L_min_s']]
vis_ks_saving_rate_mean = aac_word_sen_bei_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_ks_saving_rate_mean = vis_ks_saving_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_ks_saving_rate_mean.pivot('L_min_w', 'L_min_s', 'intuitiveness_index_T')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)
cbar.ax.set_yticklabels(["{:.1%}".format(i) for i in cbar.get_ticks()])

ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'352_aac_Word_Sen_BEI_L_L_UFI_T.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)

## 4. Experiment 3: Apply real user parameter back to model

### 4.1. Participant 1

In [ ]:
# Real user P1
T_key =  0.25 
T_react_w = 0.75 
T_react_s = 2.18 

# Prediction settings
P_pred_w_c = 0.712 # empirical data
P_pred_w_n = 0.575 # empirical data
P_pred_s = 0.438


#### 4.1.1. KLM-BEI

In [ ]:
R_rational = 0.863
R_error = 0.091
R_interrupt = 1.02

T_event = 3.92

def T_interrupt(T_event): # the probability of the system have correct sentence predictions
    a=0.189
    c=1.03
    return a*math.log(T_event)+c

# Text entering strategy
L_min_w = 5 # 2-10
k_look_w = 5 
p_max_w = 6
L_min_s = 3 # 2-10
k_look_s = 2
p_max_s = 6

In [ ]:
def real_1_Word_Sen_BEI_L_L():
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(2,10.1,0.1):
            data_frame = Run(L_min_w=i, k_look_w=k_look_w, p_max_w=p_max_w, L_min_s=j, k_look_s=k_look_s, p_max_s=p_max_s, R_rational=R_rational, R_error=R_error, R_interrupt=R_interrupt, T_event=T_event)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'411_real_1_Word_Sen_BEI_L_L.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results


In [ ]:
BOOL_PRINT = False
real_1_word_sen_BEI_L_L_results = real_1_Word_Sen_BEI_L_L()

In [ ]:
# Read from file
readname = f'411_real_1_Word_Sen_BEI_L_L.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
real_1_word_sen_bei_L_L_csv = pd.read_csv(fullname)

In [ ]:
# evaluate L_min_w and L_min_s to NER
real_1_word_sen_bei_L_L_data = real_1_word_sen_bei_L_L_csv[['entry_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = real_1_word_sen_bei_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'entry_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)


ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'411_real_1_Word_Sen_BEI_L_L_ER.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


In [ ]:
# evaluate L_min_w and L_min_s to KSR
real_1_word_sen_bei_L_L_data = real_1_word_sen_bei_L_L_csv[['KS_saving_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = real_1_word_sen_bei_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'KS_saving_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)
cbar.ax.set_yticklabels(["{:.1%}".format(i) for i in cbar.get_ticks()])

ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'411_real_1_Word_Sen_BEI_L_L_KSR.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


In [ ]:
# evaluate L_min_w and L_min_s to RI
real_1_word_sen_bei_L_L_data = real_1_word_sen_bei_L_L_csv[['rational_index', 'L_min_w', 'L_min_s']]
vis_rational_index_mean = real_1_word_sen_bei_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_rational_index_mean = vis_rational_index_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_rational_index_mean.pivot('L_min_w', 'L_min_s', 'rational_index')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)
cbar.ax.set_yticklabels(["{:.1%}".format(i) for i in cbar.get_ticks()])

ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'411_real_1_Word_Sen_BEI_L_L_RI.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


In [ ]:
# evaluate L_min_w and L_min_s to RI
real_1_word_sen_bei_L_L_data = real_1_word_sen_bei_L_L_csv[['error_index', 'L_min_w', 'L_min_s']]
vis_error_index_mean = real_1_word_sen_bei_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_error_index_mean = vis_error_index_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_error_index_mean.pivot('L_min_w', 'L_min_s', 'error_index')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)
cbar.ax.set_yticklabels(["{:.1%}".format(i) for i in cbar.get_ticks()])

ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'411_real_1_Word_Sen_BEI_L_L_EI.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


In [ ]:
# evaluate L_min_w and L_min_s to UFI_KS
real_1_word_sen_bei_L_L_data = real_1_word_sen_bei_L_L_csv[['intuitiveness_index_KS', 'L_min_w', 'L_min_s']]
vis_intuitiveness_index_ks_mean = real_1_word_sen_bei_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_intuitiveness_index_ks_mean = vis_intuitiveness_index_ks_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_intuitiveness_index_ks_mean.pivot('L_min_w', 'L_min_s', 'intuitiveness_index_KS')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)
cbar.ax.set_yticklabels(["{:.1%}".format(i) for i in cbar.get_ticks()])

ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'411_real_1_Word_Sen_BEI_L_L_UFI_KS.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


In [ ]:
# evaluate L_min_w and L_min_s to UFI_T
real_1_word_sen_bei_L_L_data = real_1_word_sen_bei_L_L_csv[['intuitiveness_index_T', 'L_min_w', 'L_min_s']]
vis_intuitiveness_index_t_mean = real_1_word_sen_bei_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_intuitiveness_index_t_mean = vis_intuitiveness_index_t_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_intuitiveness_index_t_mean.pivot('L_min_w', 'L_min_s', 'intuitiveness_index_T')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)
cbar.ax.set_yticklabels(["{:.1%}".format(i) for i in cbar.get_ticks()])

ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'411_real_1_Word_Sen_BEI_L_L_UFI_T.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


#### 4.1.2. KLM

In [ ]:
# Real user P1
T_key =  0.25 
T_react_w = 0.75 
T_react_s = 2.18 



# Prediction settings
P_pred_w_c = 0.712 # empirical data
P_pred_w_n = 0.575 # empirical data
P_pred_s = 0.438


In [ ]:
# KLM
R_rational = 1
R_error = 0
R_interrupt = 0

T_event = 0

def T_interrupt(T_event): # the probability of the system have correct sentence predictions
    a=0.189
    c=1.03
    return 0

# Text entering strategy
L_min_w = 5 # 2-10
k_look_w = 5 
p_max_w = 6
L_min_s = 3 # 2-10
k_look_s = 2
p_max_s = 6

In [ ]:
def real_1_Word_Sen_KLM_L_L():
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(2,10.1,0.1):
            data_frame = Run(L_min_w=i, k_look_w=k_look_w, p_max_w=p_max_w, L_min_s=j, k_look_s=k_look_s, p_max_s=p_max_s, R_rational=R_rational, R_error=R_error, R_interrupt=R_interrupt, T_event=T_event)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'412_real_1_Word_Sen_KLM_L_L.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results

In [ ]:
BOOL_PRINT = False
real_1_word_sen_KLM_L_L_results = real_1_Word_Sen_KLM_L_L()

In [ ]:
# Read from file
readname = f'412_real_1_Word_Sen_KLM_L_L.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
real_1_word_sen_klm_L_L_csv = pd.read_csv(fullname)

In [ ]:
# evaluate L_min_w and L_min_s to NER
real_1_word_sen_klm_L_L_data = real_1_word_sen_klm_L_L_csv[['entry_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = real_1_word_sen_klm_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'entry_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)


ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'412_real_1_Word_Sen_KLM_L_L_NER.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


In [ ]:
# evaluate L_min_w and L_min_s to KSR
real_1_word_sen_klm_L_L_data = real_1_word_sen_klm_L_L_csv[['KS_saving_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = real_1_word_sen_klm_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'KS_saving_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)
cbar.ax.set_yticklabels(["{:.1%}".format(i) for i in cbar.get_ticks()])


ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'412_real_1_Word_Sen_KLM_L_L_KSR.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


### 4.2. Participant 2

In [ ]:
# Real user P2
T_key =  0.32 
T_react_w = 0.77 
T_react_s = 1.18 

# Prediction settings
P_pred_w_c = 0.712 # empirical data
P_pred_w_n = 0.575 # empirical data
P_pred_s = 0.438


#### 4.2.1. KLM-BEI

In [ ]:
R_rational = 0.878
R_error = 0.029
R_interrupt = 1.07

T_event = 3.21

def T_interrupt(T_event): # the probability of the system have correct sentence predictions
    a=0.189
    c=1.03
    return a*math.log(T_event)+c

# Text entering strategy
L_min_w = 4 # 2-10
k_look_w = 4 
p_max_w = 7
L_min_s = 5 # 2-10
k_look_s = 4
p_max_s = 7

In [ ]:
def real_2_Word_Sen_BEI_L_L():
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(2,10.1,0.1):
            data_frame = Run(L_min_w=i, k_look_w=k_look_w, p_max_w=p_max_w, L_min_s=j, k_look_s=k_look_s, p_max_s=p_max_s)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'421_real_2_Word_Sen_BEI_L_L.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results


In [ ]:
BOOL_PRINT = False
real_2_word_sen_BEI_L_L_results = real_2_Word_Sen_BEI_L_L()

In [ ]:
# Read from file
readname = f'421_real_2_Word_Sen_BEI_L_L.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
real_2_word_sen_bei_L_L_csv = pd.read_csv(fullname)

In [ ]:
# evaluate L_min_w and L_min_s to ER
real_2_word_sen_bei_L_L_data = real_2_word_sen_bei_L_L_csv[['entry_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = real_2_word_sen_bei_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'entry_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)


ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'421_real_2_Word_Sen_BEI_L_L_ER.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


In [ ]:
# evaluate L_min_w and L_min_s to KSR
real_2_word_sen_bei_L_L_data = real_2_word_sen_bei_L_L_csv[['KS_saving_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = real_2_word_sen_bei_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'KS_saving_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)
cbar.ax.set_yticklabels(["{:.1%}".format(i) for i in cbar.get_ticks()])

ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'421_real_2_Word_Sen_BEI_L_L_KSR.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


#### 4.2.2. KLM

In [ ]:
R_rational = 1
R_error = 0
R_interrupt = 0

T_event = 5.41

def T_interrupt(T_event): # the probability of the system have correct sentence predictions
    a=0.189
    c=1.03
    return 0

# Text entering strategy
L_min_w = 4 # 2-10
k_look_w = 4 
p_max_w = 7
L_min_s = 5 # 2-10
k_look_s = 4
p_max_s = 7

In [ ]:
def real_2_Word_Sen_KLM_L_L():
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(2,10.1,0.1):
            data_frame = Run(L_min_w=i, k_look_w=k_look_w, p_max_w=p_max_w, L_min_s=j, k_look_s=k_look_s, p_max_s=p_max_s)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'422_real_2_Word_Sen_KLM_L_L.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results

In [ ]:
BOOL_PRINT = False
real_2_word_sen_KLM_L_L_results = real_2_Word_Sen_KLM_L_L()

In [ ]:
# Read from file
readname = f'422_real_2_Word_Sen_KLM_L_L.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
real_2_word_sen_klm_L_L_csv = pd.read_csv(fullname)

In [ ]:
# evaluate L_min_w and L_min_s to NER
real_2_word_sen_klm_L_L_data = real_2_word_sen_klm_L_L_csv[['entry_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = real_2_word_sen_klm_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'entry_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)


ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'422_real_2_Word_Sen_KLM_L_L_NER.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


In [ ]:
# evaluate L_min_w and L_min_s to KSR
real_2_word_sen_klm_L_L_data = real_2_word_sen_klm_L_L_csv[['KS_saving_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = real_2_word_sen_klm_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'KS_saving_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)
cbar.ax.set_yticklabels(["{:.1%}".format(i) for i in cbar.get_ticks()])

ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'422_real_2_Word_Sen_KLM_L_L_KSR.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


### 4.3. Participant 3

In [ ]:
# Real user P3
T_key =  0.27 
T_react_w = 0.98 
T_react_s = 0.87 

# Prediction settings
P_pred_w_c = 0.712 # empirical data
P_pred_w_n = 0.575 # empirical data
P_pred_s = 0.438


#### 4.3.1. KLM-BEI

In [ ]:
R_rational = 0.888
R_error = 0.060
R_interrupt = 0.08

T_event = 6.21

def T_interrupt(T_event): # the probability of the system have correct sentence predictions
    a=0.189
    c=1.03
    return a*math.log(T_event)+c

# Text entering strategy
L_min_w = 4 # 2-10
k_look_w = 1 
p_max_w = 6
L_min_s = 5 # 2-10
k_look_s = 1
p_max_s = 6

In [ ]:
def real_3_Word_Sen_BEI_L_L():
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(2,10.1,0.1):
            data_frame = Run(L_min_w=i, k_look_w=k_look_w, p_max_w=p_max_w, L_min_s=j, k_look_s=k_look_s, p_max_s=p_max_s)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'431_real_3_Word_Sen_BEI_L_L.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results


In [ ]:
BOOL_PRINT = False
real_3_word_sen_BEI_L_L_results = real_3_Word_Sen_BEI_L_L()

In [ ]:
# Read from file
readname = f'431_real_3_Word_Sen_BEI_L_L.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
real_3_word_sen_bei_L_L_csv = pd.read_csv(fullname)

In [ ]:
# evaluate L_min_w and L_min_s to NER
real_3_word_sen_bei_L_L_data = real_3_word_sen_bei_L_L_csv[['net_entry_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = real_3_word_sen_bei_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'net_entry_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)


ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'431_real_3_Word_Sen_BEI_L_L_NER.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


In [ ]:
# evaluate L_min_w and L_min_s to KSR
real_3_word_sen_bei_L_L_data = real_3_word_sen_bei_L_L_csv[['KS_saving_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = real_3_word_sen_bei_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'KS_saving_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)


ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'431_real_3_Word_Sen_BEI_L_L_KSR.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


#### 4.3.2. KLM

In [ ]:
R_rational = 1
R_error = 0
R_interrupt = 0

T_event = 5.41

def T_interrupt(T_event): # the probability of the system have correct sentence predictions
    a=0.189
    c=1.03
    return 0

# Text entering strategy
L_min_w = 4 # 2-10
k_look_w = 1 
p_max_w = 6
L_min_s = 5 # 2-10
k_look_s = 1
p_max_s = 6

In [ ]:
def real_3_Word_Sen_KLM_L_L():
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(2,10.1,0.1):
            data_frame = Run(L_min_w=i, k_look_w=k_look_w, p_max_w=p_max_w, L_min_s=j, k_look_s=k_look_s, p_max_s=p_max_s)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'432_real_3_Word_Sen_KLM_L_L.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results

In [ ]:
BOOL_PRINT = False
real_3_word_sen_KLM_L_L_results = real_3_Word_Sen_KLM_L_L()

In [ ]:
# Read from file
readname = f'432_real_3_Word_Sen_KLM_L_L.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
real_3_word_sen_klm_L_L_csv = pd.read_csv(fullname)

In [ ]:
# evaluate L_min_w and L_min_s to NER
real_3_word_sen_klm_L_L_data = real_3_word_sen_klm_L_L_csv[['net_entry_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = real_3_word_sen_klm_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'net_entry_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)


ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'432_real_3_Word_Sen_KLM_L_L_NER.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


In [ ]:
# evaluate L_min_w and L_min_s to KSR
real_3_word_sen_klm_L_L_data = real_3_word_sen_klm_L_L_csv[['KS_saving_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = real_3_word_sen_klm_L_L_data.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'KS_saving_rate')

plt.xlabel('L_min_s', size=25)
plt.ylabel('L_min_w', size=25)
im = plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
cbar = plt.colorbar(im)
cbar.ax.tick_params(labelsize=25)
print(pivot_table.values.max)


ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
ax.tick_params(axis='both', which='major', labelsize=25)
# ax.tick_params(axis='both', which='minor', labelsize=20)
# plt.colorbar()

# Save data
outname = f'432_real_3_Word_Sen_KLM_L_L_KSR.csv'
outdir = './analysis_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


# Legacy

## Data collection

### Load function: Randomly collect samples from daily conversation dataset with AAC

In [ ]:
def No_duplicate_words(word_list):
    word_list_lower = [x.lower() for x in word_list]
    if len(set(word_list_lower)) < len(word_list_lower):
        return False
    return True   

In [ ]:
def Take_sample_from_original_source(sample_size):
    with open('sent_train_aac.txt') as file:
        lines = file.readlines()

    sentence_samples = []
    average_sentence_length = 0
    for i in range(sample_size):
        sample = int(random.random()*len(lines))
        line_temp = lines[sample].strip()
        line_temp = re.sub(r'[^\w\s]', '', line_temp)
        line_temp_list = line_temp.split(' ')
        
        if No_duplicate_words(line_temp_list):
            sentence_samples.append(lines[sample])
        else: 
            BOOL_DUPLICATE = True
            if sample < len(lines):
                sample_temp = sample + 1
            else:
                if sample > 10:
                    sample_temp = sample - 10
                else:
                    sample_temp = 10
            while BOOL_DUPLICATE:
                # print(sample_temp)
                line_temp = lines[sample_temp].strip()
                line_temp_list = line_temp.split(' ')
                if No_duplicate_words(line_temp_list):
                    sentence_samples.append(lines[sample_temp])
                    BOOL_DUPLICATE = False
                    break
                else:
                    sample_temp += 1
                
        if BOOL_PRINT:
            print(f"{i+1}. Select sample {sample}: {lines[sample]}")

    with open(f'aac_daily_conversation_sample_{sample_size}.txt', 'w') as file:
        for sample in sentence_samples:
            word_list = sample.split(' ')
            average_sentence_length += len(word_list)
            file.write("%s" % sample) 

    average_sentence_length = average_sentence_length/len(sentence_samples)
    print(f"Average sentence length: {average_sentence_length}")
    return average_sentence_length

### Execution: Data collection

In [ ]:
SAMPLE_SIZE = 100
L_s_ave = Take_sample_from_original_source(SAMPLE_SIZE)

## Exam data validity

In [ ]:
SAMPLE_SIZE = 100

# Word prediction settings
P_pred_w_c = 0.712
P_pred_w_n = 0.575
P_pred_s = 0.438

# Sentence prediction settings
# def P_pred_s(current_word_index, L_s): # the probability of the system have correct sentence predictions
#     a=1
#     b=L_s
#     x=current_word_index
#     c=0
#     return a*math.log(x+1,b)+c

# User profile
T_key = 0.26 #2.5
T_react_w = 0.45
T_react_s = 1.90 # T_react_w * average_sentence_length

R_rational = 0.9
R_error = 0.1

R_interrupt = 0.1
T_event = 60*random.random()
def T_interrupt(T_event): # the probability of the system have correct sentence predictions
    a=0.189
    c=1.03
    return a*math.log(T_event)+c

# Text entering strategy
L_min_w = 1
k_look_w = 1
p_max_w = 4

L_min_s = 2
k_look_s = 1
p_max_s = 6


# with open(f'aac_daily_conversation_sample_{SAMPLE_SIZE}.txt') as file:
with open(f'raw_data.txt') as file:
    samples = file.readlines()
# samples = ["Oh so what did dad have to say to you?"]
data_dic_list = []

for i, s in enumerate(samples):
    # print(f"In sample {i}/{len(samples)}: {s}")
    sample = s.strip()
    word_list = sample.split(' ')
    dic_entry_record, T_total, KS_total = Type_one_sentence(sample, L_min_w, k_look_w, p_max_w, L_min_s, k_look_s, p_max_s) # Take_average_of_one_sentence_result(1, sample)
    
    data_dic = {
        "L_min_w": float(L_min_w),
        "k_look_w": float(k_look_w),
        "p_max_w": float(p_max_w),
        "L_min_s": float(L_min_s),
        "k_look_s": float(k_look_s),
        "p_max_s": float(p_max_s),
        "P_pred_w_c": float(P_pred_w_c),
        "P_pred_w_n": float(P_pred_w_n),
        "T_key": float(T_key),
        "T_react_w": float(T_react_w),
        "T_react_s": float(T_react_s),
        "R_rational": float(R_rational), # bounded rationality index
        "R_error": float(R_error), # human error rate
        "R_interrupt": float(R_interrupt), # interruption rate
        "T_event": float(T_event), # interruption time
        # "T_interrupt": 
        
        "sentence": sample,
        "sentence_length": int(len(sample)),
        "sentence_length_in_words": int(len(word_list)),
        "T_total": float(T_total),
        "KS_total": float(KS_total),
        "entry_rate": float(KS_total/T_total)*12.0, # wpm
        # "effective_entry_rate":
        "net_entry_rate": float((KS_total/T_total)*12.0 - (12.0/T_key)), # wpm
        # "net_effective_entry_rate":
        "KS_saving": float(float(len(sample)) - KS_total),
        "KS_saving_rate": float(float(len(sample)) - KS_total)/(float(len(sample)))*100 # %
    }
    data_dic_list.append(data_dic)
df_data_dic_list = pd.DataFrame(data=data_dic_list)
df_data_dic_list.to_excel('exam-data-validity.xlsx')

net_entry_rate_mean = df_data_dic_list.groupby(['sentence_length_in_words'])['net_entry_rate'].mean()
df_data_dic_list.plot.scatter(x='sentence_length' ,y='sentence_length_in_words')
print(net_entry_rate_mean)

## Experiment 1: KLM - Sentence generation system
Run experiment under error-free, interruption-free and rational conditions. Aim to evaluate the efficiency of the system design with different entry strategies for the AAC user. User data from (Koester, 1994).

### Load fixed parameters for AAC users

In [ ]:
# User profile
T_key =  0.26 # 0.26 # 0.6
T_react_w = 0.45 # 0.45 # 1.20
T_react_s = 1.90 # 1.90 # T_react_w * L_s_ave = 1.20*4.23 = 5.08

R_rational = 1
R_error = 0
R_interrupt = 0

T_event = 5

def T_interrupt(T_event): # the probability of the system have correct sentence predictions
    a=0.189
    c=1.03
    return a*math.log(T_event)+c

# Word prediction settings
P_pred_w_c = 0.712 # 0.6 # KWickChat
P_pred_w_n = 0.575 # 0.5 # KWickChat
P_pred_s = 0.438

# Sentence prediction settings
# def P_pred_s(current_word_index, L_s): # the probability of the system have correct sentence predictions
#     a=1
#     b=L_s
#     x=current_word_index
#     c=0
#     return a*math.log(x+1,b)+c

### Load initial variables: entry without assistance

In [ ]:
# Text entering strategy
L_min_w = 100
k_look_w = 0
p_max_w = 0

L_min_s = 100
k_look_s = 0
p_max_s = 0

### Load fundamental function

In [ ]:
# def sort_by_sentence_length(e):
#     return e["sentence_length_in_words"]
    
def Run(SAMPLE_SIZE, CONDITION, L_min_w=100.0, k_look_w=0.0, p_max_w=0.0, L_min_s=100.0, k_look_s=0.0, p_max_s=0.0, R_rational=1.0, R_error=0.0, R_interrupt=0.0, T_event=5.0):
    with open(f'aac_daily_conversation_sample_{SAMPLE_SIZE}.txt') as file:
        samples = file.readlines()
    # samples = ["Oh so what did dad have to say to you?"]
    data_dic_list = []

    for i, s in enumerate(samples):
        # print(f"In sample {i}/{len(samples)}: {s}")
        sample = s.strip()
        sample = re.sub(r'[^\w\s]', '', sample)
        word_list = sample.split(' ')
        dic_entry_record, T_total, KS_total = Type_one_sentence(sample, L_min_w, k_look_w, p_max_w, L_min_s, k_look_s, p_max_s, R_rational, R_error, R_interrupt, T_event) # Take_average_of_one_sentence_result(1, sample)
        data_dic = {
            # settings
            "L_min_w": float(L_min_w),
            "k_look_w": float(k_look_w),
            "p_max_w": float(p_max_w),
            "L_min_s": float(L_min_s),
            "k_look_s": float(k_look_s),
            "p_max_s": float(p_max_s),
            "P_pred_w_c": float(P_pred_w_c),
            "P_pred_w_n": float(P_pred_w_n),
            "T_key": float(T_key),
            "T_react_w": float(T_react_w),
            "T_react_s": float(T_react_s),
            "R_rational": float(R_rational), # bounded rationality index
            "R_error": float(R_error), # human error rate
            "R_interrupt": float(R_interrupt), # interruption rate
            "T_event": float(T_event), # interruption time

            # results
            "sentence": sample,
            "sentence_length": int(len(sample)),
            "sentence_length_in_words": int(len(word_list)),

            "T_total": dic_entry_record['T_total'],
            "T_key_total": dic_entry_record['T_key_total'],
            "T_react_w_total": dic_entry_record['T_react_w_total'],
            "T_react_s_total": dic_entry_record['T_react_s_total'],
            "T_react_total": dic_entry_record['T_react_total'],
            "T_bounded_l_total": dic_entry_record['T_bounded_l_total'],
            "T_bounded_w_total": dic_entry_record['T_bounded_w_total'],
            "T_bounded_s_total": dic_entry_record['T_bounded_s_total'],
            "T_error_total": dic_entry_record['T_error_total'],
            "T_interrupt_total": dic_entry_record['T_interrupt_total'],

            "T_event_total": dic_entry_record['T_event_total'],
            "N_bounded_l_total": dic_entry_record['N_bounded_l_total'],
            "N_bounded_w_total": dic_entry_record['N_bounded_w_total'], 
            "N_bounded_s_total": dic_entry_record['N_bounded_s_total'], 
            "N_bounded_total": dic_entry_record['N_bounded_total'],

            "KS_total": dic_entry_record['KS_total'],
            "KS_type_total": dic_entry_record['KS_type_total'],
            "KS_select_total": dic_entry_record['KS_select_total'],
            "KS_bounded_l_total": dic_entry_record['KS_bounded_l_total'],
            "KS_bounded_w_total": dic_entry_record['KS_bounded_w_total'],
            "KS_bounded_s_total": dic_entry_record['KS_bounded_s_total'],
            "KS_error_total": dic_entry_record['KS_error_total'],
            "KS_interrupt_total": dic_entry_record['KS_interrupt_total'],

            # basic statistics
            "entry_rate": float(len(sample)/T_total)*12.0, # wpm
            # "effective_entry_rate":
            "net_entry_rate": float((len(sample)/T_total)*12.0 - (12.0/T_key)), # wpm
            # "net_effective_entry_rate":
            "KS_saving": float(float(len(sample)) - KS_total),
            "KS_saving_rate": float(float(len(sample)) - KS_total)/float(len(sample)) # %
        }

        data_dic_list.append(data_dic)

    # data_dic_list.sort(key=sort_by_sentence_length)

    # Collect data
    df_data_dic_list = pd.DataFrame(data=data_dic_list)

    return df_data_dic_list

### TODO: test Run()

### Run condition A: No prediction

#### Load initial parameters and functions

In [ ]:
# Text entering strategy
L_min_w = 100
k_look_w = 0
p_max_w = 0
L_min_s = 100
k_look_s = 0
p_max_s = 0

In [ ]:
def Run_condition_A_all_data():
    exp_result_dic_list = []
    condition = "A"
    for i in np.arange(2,10): # .1,0.1
        for j in np.arange(0,4): # .1,0.1
            data_frame = Run(SAMPLE_SIZE=100, CONDITION=condition, L_min_w=i, k_look_w=j)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'experiment_1_{condition}.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results

#### Condition A: Execution

In [ ]:
df_exp_results = Run_condition_A_all_data()

In [ ]:
print(df_exp_results)

#### Condition A: Analysis

### Run condition AB: Word prediction only

#### Load initial parameters and functions

In [ ]:
# Text entering strategy
L_min_w = 2 # 2-10
k_look_w = 1 # 0-5
p_max_w = 6
L_min_s = 100
k_look_s = 0
p_max_s = 0

In [ ]:
def Compare_condition_AB_all_data(condition):
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(0,5.1,0.1):
            data_frame = Run(SAMPLE_SIZE=100, CONDITION=condition, L_min_w=i, k_look_w=j, p_max_w=p_max_w)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'experiment_1_{condition}.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results

In [ ]:
def Compare_condition_AB_p_L_all_data(condition):
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(2,10.1,0.1):
            data_frame = Run(SAMPLE_SIZE=100, CONDITION=condition, L_min_w=i, k_look_w=1, p_max_w=j)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'experiment_1_{condition}.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results

#### Condition AB: Execution

In [ ]:
BOOL_PRINT = False
con_ab_df_exp_results = Compare_condition_AB_all_data('AB')

In [ ]:
BOOL_PRINT = False
con_ab_df_exp_results = Compare_condition_AB_all_data('AB_current_w_only')

In [ ]:
BOOL_PRINT = False
con_ab_df_exp_results = Compare_condition_AB_all_data('AB_next_w_only')

In [ ]:
BOOL_PRINT = False
con_ab_df_exp_results = Compare_condition_AB_all_data('AB_current_w_only_ablebodied')

In [ ]:
BOOL_PRINT = False
con_ab_p_L_df_exp_results = Compare_condition_AB_p_L_all_data('AB_p_L')

#### Condition AB: Analysis

##### Read from file

In [ ]:
# Read from file
condition = 'AB'
readname = f'experiment_1_{condition}.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
con_ab_df_exp_results_csv = pd.read_csv(fullname)

In [ ]:
# Read from file
condition = 'AB_current_w_only'
readname = f'experiment_1_{condition}.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
con_ab_c_only_df_exp_results_csv = pd.read_csv(fullname)

In [ ]:
# Read from file
condition = 'AB_next_w_only'
readname = f'experiment_1_{condition}.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
con_ab_n_only_df_exp_results_csv = pd.read_csv(fullname)

In [ ]:
# Read from file
condition = 'AB_current_w_only_ablebodied'
readname = f'experiment_1_{condition}.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
con_ab_c_only_ablebodied_df_exp_results_csv = pd.read_csv(fullname)

In [ ]:
# Read from file
condition = 'AB_p_L'
readname = f'experiment_1_{condition}.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
con_ab_df_exp_p_L_results_csv = pd.read_csv(fullname)

##### Net_entry_rate: L_min_w & k_look_w

In [ ]:
# evaluate L_min_w and k_look_w
con_ab_net_entry_rate_L_min_w_k_look_w = con_ab_df_exp_results_csv[['net_entry_rate', 'L_min_w', 'k_look_w']]
vis_net_entry_rate_mean = con_ab_net_entry_rate_L_min_w_k_look_w.groupby(['L_min_w', 'k_look_w']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

# vis_net_entry_rate_std = con_ab_net_entry_rate_L_min_w_k_look_w.groupby(['L_min_w', 'k_look_w']).std()
# vis_net_entry_rate_std = vis_net_entry_rate_std.reset_index()


fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('k_look_w', 'L_min_w', 'net_entry_rate')
# pivot_table_std = vis_net_entry_rate_std.pivot('k_look_w', 'L_min_w', 'net_entry_rate')
# print(pivot_table.shape)
# print(pivot_table_std.shape)


plt.xlabel('L_min_w', size=15)
plt.ylabel('k_look_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(pivot_table_std)), cmap='coolwarm', interpolation='nearest') # x_stddev=.3,y_stddev=.3
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(0, 5, 6, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()

# Save data
outname = f'exp1_L_k_ner.csv'
outdir = './experiment_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)
            

In [ ]:
# evaluate L_min_w and k_look_w for current word pred only
con_ab_net_entry_rate_L_min_w_k_look_w = con_ab_c_only_df_exp_results_csv[['net_entry_rate', 'L_min_w', 'k_look_w']]
vis_net_entry_rate_mean = con_ab_net_entry_rate_L_min_w_k_look_w.groupby(['L_min_w', 'k_look_w']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

# vis_net_entry_rate_std = con_ab_net_entry_rate_L_min_w_k_look_w.groupby(['L_min_w', 'k_look_w']).std()
# vis_net_entry_rate_std = vis_net_entry_rate_std.reset_index()


fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('k_look_w', 'L_min_w', 'net_entry_rate')
# pivot_table_std = vis_net_entry_rate_std.pivot('k_look_w', 'L_min_w', 'net_entry_rate')
# print(pivot_table.shape)
# print(pivot_table_std.shape)


plt.xlabel('L_min_w', size=15)
plt.ylabel('k_look_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(pivot_table_std)), cmap='coolwarm', interpolation='nearest') # x_stddev=.3,y_stddev=.3
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(0, 5, 6, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()

# Save data
outname = f'exp1_AB_current_w_only_ablebodied_ner.csv'
outdir = './experiment_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)

In [ ]:
# evaluate L_min_w and k_look_w for next word pred only
con_ab_net_entry_rate_L_min_w_k_look_w = con_ab_n_only_df_exp_results_csv[['net_entry_rate', 'L_min_w', 'k_look_w']]
vis_net_entry_rate_mean = con_ab_net_entry_rate_L_min_w_k_look_w.groupby(['L_min_w', 'k_look_w']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

# vis_net_entry_rate_std = con_ab_net_entry_rate_L_min_w_k_look_w.groupby(['L_min_w', 'k_look_w']).std()
# vis_net_entry_rate_std = vis_net_entry_rate_std.reset_index()


fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('k_look_w', 'L_min_w', 'net_entry_rate')
# pivot_table_std = vis_net_entry_rate_std.pivot('k_look_w', 'L_min_w', 'net_entry_rate')
# print(pivot_table.shape)
# print(pivot_table_std.shape)


plt.xlabel('L_min_w', size=15)
plt.ylabel('k_look_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(pivot_table_std)), cmap='coolwarm', interpolation='nearest') # x_stddev=.3,y_stddev=.3
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(0, 5, 6, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()

In [ ]:
# evaluate L_min_w and k_look_w for ablebodied
con_ab_net_entry_rate_L_min_w_k_look_w = con_ab_c_only_ablebodied_df_exp_results_csv[['net_entry_rate', 'L_min_w', 'k_look_w']]
vis_net_entry_rate_mean = con_ab_net_entry_rate_L_min_w_k_look_w.groupby(['L_min_w', 'k_look_w']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

# vis_net_entry_rate_std = con_ab_net_entry_rate_L_min_w_k_look_w.groupby(['L_min_w', 'k_look_w']).std()
# vis_net_entry_rate_std = vis_net_entry_rate_std.reset_index()


fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('k_look_w', 'L_min_w', 'net_entry_rate')
# pivot_table_std = vis_net_entry_rate_std.pivot('k_look_w', 'L_min_w', 'net_entry_rate')
# print(pivot_table.shape)
# print(pivot_table_std.shape)


plt.xlabel('L_min_w', size=15)
plt.ylabel('k_look_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(pivot_table_std)), cmap='coolwarm', interpolation='nearest') # x_stddev=.3,y_stddev=.3
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(0, 5, 6, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()

##### Net_entry_rate_std: L_min_w & k_look_w

In [ ]:
# evaluate L_min_w and k_look_w
con_ab_net_entry_rate_L_min_w_k_look_w = con_ab_df_exp_results_csv[['net_entry_rate', 'L_min_w', 'k_look_w']]
vis_net_entry_rate_std = con_ab_net_entry_rate_L_min_w_k_look_w.groupby(['L_min_w', 'k_look_w']).std()
vis_net_entry_rate_std = vis_net_entry_rate_std.reset_index()

# vis_net_entry_rate_std = con_ab_net_entry_rate_L_min_w_k_look_w.groupby(['L_min_w', 'k_look_w']).std()
# vis_net_entry_rate_std = vis_net_entry_rate_std.reset_index()


fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_std.pivot('k_look_w', 'L_min_w', 'net_entry_rate')



plt.xlabel('L_min_w', size=15)
plt.ylabel('k_look_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(pivot_table_std)), cmap='coolwarm', interpolation='nearest') # x_stddev=.3,y_stddev=.3
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(0, 5, 6, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()

##### p_max_w VS L_min_w

In [ ]:


# evaluate L_min_w and p_max_w
con_ab_net_entry_rate_L_min_w_k_look_w = con_ab_df_exp_p_L_results_csv[['net_entry_rate', 'L_min_w', 'p_max_w']]
vis_net_entry_rate_mean = con_ab_net_entry_rate_L_min_w_k_look_w.groupby(['L_min_w', 'p_max_w']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

# vis_net_entry_rate_std = con_ab_net_entry_rate_L_min_w_k_look_w.groupby(['L_min_w', 'k_look_w']).std()
# vis_net_entry_rate_std = vis_net_entry_rate_std.reset_index()


fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('p_max_w', 'L_min_w', 'net_entry_rate')
# pivot_table_std = vis_net_entry_rate_std.pivot('k_look_w', 'L_min_w', 'net_entry_rate')
# print(pivot_table.shape)
# print(pivot_table_std.shape)


plt.xlabel('L_min_w', size=15)
plt.ylabel('p_max_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(pivot_table_std)), cmap='coolwarm', interpolation='nearest') # x_stddev=.3,y_stddev=.3
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()

# table = pd.pivot_table(pivot_table, index='p_max_w', values='L_min_w', aggfunc=[np.max, np.min])

# print(f"{pivot_table}")


##### For able-bodied users

In [ ]:
# User profile
T_key =  0.26 # 0.26 # 0.6
T_react_w = 0.45 # 0.45 # 1.20
T_react_s = 1.90 # T_react_w * L_s_ave = 1.20*4.23 = 5.08

R_rational = 1
R_error = 0
R_interrupt = 0

T_event = 0

def T_interrupt(T_event): # the probability of the system have correct sentence predictions
    a=0.189
    c=1.03
    return 0

# Word prediction settings
P_pred_w_c = 1 # 0.6 # KWickChat
P_pred_w_n = 1 # 0.5 # KWickChat

# Sentence prediction settings
def P_pred_s(current_word_index, L_s): # the probability of the system have correct sentence predictions
    a=1
    b=L_s
    x=current_word_index
    c=0
    return a*math.log(x+1,b)+c

# Text entering strategy
L_min_w = 2 # 2-10
k_look_w = 0 # 0-5
p_max_w = 6
L_min_s = 100
k_look_s = 0
p_max_s = 0

In [ ]:
con_ab_df_exp_results_ablebodied = Compare_condition_AB_all_data('AB_ablebodied')

In [ ]:
# evaluate L_min_w and k_look_w
con_ab_net_entry_rate_L_min_w_k_look_w = con_ab_df_exp_results_ablebodied[['net_entry_rate', 'L_min_w', 'k_look_w']]
vis_net_entry_rate_mean = con_ab_net_entry_rate_L_min_w_k_look_w.groupby(['L_min_w', 'k_look_w']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('k_look_w', 'L_min_w', 'net_entry_rate')

plt.xlabel('L_min_w', size=15)
plt.ylabel('k_look_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(pivot_table_std)), cmap='coolwarm', interpolation='nearest') # x_stddev=.3,y_stddev=.3
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(0, 5, 6, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()


##### net_entry_rate: sentence_length_in_words

In [ ]:
# evaluate net_entry_rate and sentence_length_in_words
con_ab_net_entry_rate_L_min_w_k_look_w = con_ab_df_exp_results_csv[['net_entry_rate', 'L_min_w', 'sentence_length_in_words']]
con_ab_net_entry_rate_mean_L_min_w = con_ab_net_entry_rate_L_min_w_k_look_w.groupby(['L_min_w', 'sentence_length_in_words']).mean()
vis_net_entry_rate_mean = con_ab_net_entry_rate_mean_L_min_w.groupby(['sentence_length_in_words']).mean()
# vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
# pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'sentence_length', 'net_entry_rate')
plt.xlabel('sentence_length_in_words', size=15)
plt.ylabel('net_entry_rate_mean', size=15)
plt.plot(vis_net_entry_rate_mean)
plt.axhline(y=0, color='k', linestyle='--')
plt.show()


##### KS_saving_rate: L_min_w & k_look_w

In [ ]:
# evaluate L_min_w and k_look_w
con_ab_KS_saving_rate_L_min_w_k_look_w = con_ab_df_exp_results_csv[['KS_saving_rate', 'L_min_w', 'k_look_w']]
vis_KS_saving_rate_mean = con_ab_KS_saving_rate_L_min_w_k_look_w.groupby(['L_min_w', 'k_look_w']).mean()
vis_KS_saving_rate_mean = vis_KS_saving_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_KS_saving_rate_mean.pivot('k_look_w', 'L_min_w', 'KS_saving_rate')
plt.xlabel('L_min_w', size=15)
plt.ylabel('k_look_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.4,y_stddev=.4)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(0, 5, 6, dtype=np.int16)
ax.set_yticklabels(yticks)
cb = plt.colorbar()
cb.ax.set_yticklabels(["{:.1%}".format(i) for i in cb.get_ticks()])
# plt.colorbar(['{:.0f}%'.format(x) for x in np.arange(vis_KS_saving_rate_mean['KS_saving_rate'].min(), vis_KS_saving_rate_mean['KS_saving_rate'].max(), 10)])


##### KS_saving_rate_std: L_min_w & k_min_w

In [ ]:
# evaluate L_min_w and k_look_w
con_ab_KS_saving_rate_L_min_w_k_look_w = con_ab_df_exp_results_csv[['KS_saving_rate', 'L_min_w', 'k_look_w']]
vis_KS_saving_rate_std = con_ab_KS_saving_rate_L_min_w_k_look_w.groupby(['L_min_w', 'k_look_w']).std()
vis_KS_saving_rate_std = vis_KS_saving_rate_std.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_KS_saving_rate_std.pivot('k_look_w', 'L_min_w', 'KS_saving_rate')
plt.xlabel('L_min_w', size=15)
plt.ylabel('k_look_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.4,y_stddev=.4)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(0, 5, 6, dtype=np.int16)
ax.set_yticklabels(yticks)
cb = plt.colorbar()
cb.ax.set_yticklabels(["{:.1%}".format(i) for i in cb.get_ticks()])


##### KS_saving_rate & sentence_length_in_words

In [ ]:
# evaluate KS_saving_rate and sentence_length_in_words
con_ab_KS_saving_rate_L_min_w_k_look_w = con_ab_df_exp_results_csv[['KS_saving_rate', 'L_min_w', 'sentence_length_in_words']]
con_ab_KS_saving_rate_mean_L_min_w = con_ab_KS_saving_rate_L_min_w_k_look_w.groupby(['L_min_w', 'sentence_length_in_words']).mean()
vis_KS_saving_rate_mean = con_ab_KS_saving_rate_mean_L_min_w.groupby(['sentence_length_in_words']).mean()
# vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
# pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'sentence_length', 'net_entry_rate')
plt.xlabel('sentence_length_in_words', size=15)
plt.ylabel('KS_saving_rate_mean (%)', size=15)
plt.plot(vis_KS_saving_rate_mean, label="KS saving rate")
KS_saving_rate_mean_mean = vis_KS_saving_rate_mean['KS_saving_rate'].mean()
# print(vis_KS_saving_rate_mean)
plt.axhline(y=KS_saving_rate_mean_mean, color='k', linestyle='--', label='average KS saving rate')
plt.legend(loc = 'upper right')

plt.show()

### Condition AC: Sentence prediction only

#### Load initial parameters and functions

In [ ]:
# Text entering strategy
L_min_w = 100 
k_look_w = 0 
p_max_w = 0
L_min_s = 2 # 2-10
k_look_s = 2 # 0-5
p_max_s = 6 # 8

In [ ]:
def Compare_condition_AC_all_data(condition):
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(0,5.1,0.1):
            data_frame = Run(SAMPLE_SIZE=100, CONDITION=condition, L_min_s=i, k_look_s=j, p_max_s=6)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'experiment_1_{condition}.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results

In [ ]:
def Compare_condition_AC_p_L_all_data(condition):
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(2,10.1,0.1):
            data_frame = Run(SAMPLE_SIZE=100, CONDITION=condition, L_min_s=i, k_look_s=1, p_max_s=j)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'experiment_1_{condition}.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results

#### Condition AC: Execution

In [ ]:
BOOL_PRINT = False
con_ac_df_exp_results = Compare_condition_AC_all_data('AC')

In [ ]:
BOOL_PRINT = False
con_ac_df_exp_p_L_results_csv = Compare_condition_AC_p_L_all_data('AC_p_L')

#### Condition AC: Analysis

##### Read from file

In [ ]:
# Read from file
condition = 'AC'
readname = f'experiment_1_{condition}.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
con_ac_df_exp_results_csv = pd.read_csv(fullname)

In [ ]:
# Read from file
condition = 'AC_p_L'
readname = f'experiment_1_{condition}.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
con_ac_df_exp_p_L_results_csv = pd.read_csv(fullname)

##### L_min_s & k_look_s

In [ ]:
# evaluate L_min_w and k_look_w
con_ac_net_entry_rate_L_min_s_k_look_s = con_ac_df_exp_results_csv[['net_entry_rate', 'L_min_s', 'k_look_s']]
vis_net_entry_rate_mean = con_ac_net_entry_rate_L_min_s_k_look_s.groupby(['L_min_s', 'k_look_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('k_look_s', 'L_min_s', 'net_entry_rate')
plt.xlabel('L_min_s', size=15)
plt.ylabel('k_look_s', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.4,y_stddev=.4)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(0, 5, 6, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()


In [ ]:
# evaluate L_min_w and k_look_w
con_ac_net_entry_rate_L_min_s_k_look_s = con_ac_df_exp_results_csv[['KS_saving_rate', 'L_min_s', 'k_look_s']]
vis_net_entry_rate_mean = con_ac_net_entry_rate_L_min_s_k_look_s.groupby(['L_min_s', 'k_look_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('k_look_s', 'L_min_s', 'KS_saving_rate')
plt.xlabel('L_min_s', size=15)
plt.ylabel('k_look_s', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.4,y_stddev=.4)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(0, 5, 6, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()

In [ ]:
# evaluate L_min_w and k_look_w
con_ac_net_entry_rate_L_min_s_k_look_s = con_ac_df_exp_results_csv[['net_entry_rate', 'L_min_s', 'k_look_s']]
vis_net_entry_rate_mean = con_ac_net_entry_rate_L_min_s_k_look_s.groupby(['L_min_s', 'k_look_s']).std()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('k_look_s', 'L_min_s', 'net_entry_rate')
plt.xlabel('L_min_s', size=15)
plt.ylabel('k_look_s', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.4,y_stddev=.4)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(0, 5, 6, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()



##### p_max_s & L_min_s

In [ ]:
# evaluate L_min_s and p_max_s
con_ac_net_entry_rate_L_min_s_k_look_s = con_ac_df_exp_p_L_results_csv[['net_entry_rate', 'L_min_s', 'p_max_s']]
vis_net_entry_rate_mean = con_ac_net_entry_rate_L_min_s_k_look_s.groupby(['L_min_s', 'p_max_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('p_max_s', 'L_min_s', 'net_entry_rate')
plt.xlabel('L_min_s', size=15)
plt.ylabel('p_max_s', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.4,y_stddev=.4)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()

In [ ]:
# evaluate L_min_s and p_max_s
con_ac_net_entry_rate_L_min_s_k_look_s = con_ac_df_exp_p_L_results_csv[['net_entry_rate', 'L_min_s', 'p_max_s']]
vis_net_entry_rate_mean = con_ac_net_entry_rate_L_min_s_k_look_s.groupby(['L_min_s', 'p_max_s']).std()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('p_max_s', 'L_min_s', 'net_entry_rate')
plt.xlabel('L_min_s', size=15)
plt.ylabel('p_max_s', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.4,y_stddev=.4)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()

##### net_entry_rate & sentence_length_in_words

In [ ]:
# evaluate net_entry_rate and sentence_length_in_words
con_ac_net_entry_rate_L_min_s_k_look_s = con_ac_df_exp_results_csv[['net_entry_rate', 'L_min_s', 'sentence_length_in_words']] # _in_words
con_ac_net_entry_rate_mean_L_min_s = con_ac_net_entry_rate_L_min_s_k_look_s.groupby(['L_min_s', 'sentence_length_in_words']).mean()
vis_net_entry_rate_mean = con_ac_net_entry_rate_mean_L_min_s.groupby(['sentence_length_in_words']).mean()
# vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
# pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'sentence_length', 'net_entry_rate')
plt.xlabel('sentence_length_in_words', size=15)
plt.ylabel('net_entry_rate_mean', size=15)
plt.plot(vis_net_entry_rate_mean)
plt.axhline(y=0, color='k', linestyle='--')
plt.show()


##### net_entry_rate & sentence_length

In [ ]:
# evaluate net_entry_rate and sentence_length_in_words
con_ac_net_entry_rate_L_min_s_k_look_s = con_ac_df_exp_results_csv[['net_entry_rate', 'L_min_s', 'sentence_length']] # _in_words
con_ac_net_entry_rate_mean_L_min_s = con_ac_net_entry_rate_L_min_s_k_look_s.groupby(['L_min_s', 'sentence_length']).mean()
vis_net_entry_rate_mean = con_ac_net_entry_rate_mean_L_min_s.groupby(['sentence_length']).mean()
# vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
# pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'sentence_length', 'net_entry_rate')
plt.xlabel('sentence_length', size=15)
plt.ylabel('net_entry_rate_mean', size=15)
plt.plot(vis_net_entry_rate_mean)
plt.axhline(y=0, color='k', linestyle='--')
plt.show()


##### KS_saving_rate & sentence_length_in_words

In [ ]:
# evaluate KS_saving_rate and sentence_length_in_words
con_ac_KS_saving_rate_L_min_s_k_look_s = con_ac_df_exp_results_csv[['KS_saving_rate', 'L_min_s', 'sentence_length_in_words']]
con_ac_KS_saving_rate_mean_L_min_s = con_ac_KS_saving_rate_L_min_s_k_look_s.groupby(['L_min_s', 'sentence_length_in_words']).mean()
vis_KS_saving_rate_mean = con_ac_KS_saving_rate_mean_L_min_s.groupby(['sentence_length_in_words']).mean()
# vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
# pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'sentence_length', 'net_entry_rate')
plt.xlabel('sentence_length_in_words', size=15)
plt.ylabel('KS_saving_rate_mean', size=15)
plt.plot(vis_KS_saving_rate_mean, label="KS saving rate")
KS_saving_rate_mean_mean = vis_KS_saving_rate_mean['KS_saving_rate'].mean()
# print(vis_KS_saving_rate_mean)
plt.axhline(y=KS_saving_rate_mean_mean, color='k', linestyle='--', label='average KS saving rate')
plt.legend(loc = 'upper right')

plt.show()

##### KS_saving_rate & sentence_length

In [ ]:
# evaluate KS_saving_rate and sentence_length_in_words
con_ac_KS_saving_rate_L_min_s_k_look_s = con_ac_df_exp_results_csv[['KS_saving_rate', 'L_min_s', 'sentence_length']]
con_ac_KS_saving_rate_mean_L_min_s = con_ac_KS_saving_rate_L_min_s_k_look_s.groupby(['L_min_s', 'sentence_length']).mean()
vis_KS_saving_rate_mean = con_ac_KS_saving_rate_mean_L_min_s.groupby(['sentence_length']).mean()
# vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
# pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'sentence_length', 'net_entry_rate')
plt.xlabel('sentence_length', size=15)
plt.ylabel('KS_saving_rate_mean (%)', size=15)
plt.plot(vis_KS_saving_rate_mean, label="KS saving rate")
KS_saving_rate_mean_mean = vis_KS_saving_rate_mean['KS_saving_rate'].mean()
# print(vis_KS_saving_rate_mean)
plt.axhline(y=KS_saving_rate_mean_mean, color='k', linestyle='--', label='average KS saving rate')
plt.legend(loc = 'upper right')

plt.show()

### Condition AD: Word prediction + sentence prediction

#### Load initial parameters and functions

In [ ]:
# Text entering strategy
L_min_w = 4 # 2-10
k_look_w = 1 
p_max_w = 6
L_min_s = 5 # 2-10
k_look_s = 1
p_max_s = 6

In [ ]:
def Compare_condition_AD_all_data(condition):
    # condition = "AD"
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(2,10.1,0.1):
            data_frame = Run(SAMPLE_SIZE=100, CONDITION=condition, L_min_w=i, k_look_w=k_look_w, p_max_w=p_max_w, L_min_s=j, k_look_s=k_look_s, p_max_s=p_max_s)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'experiment_1_{condition}.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results

In [ ]:
def exp_1_con_ad_p_L_df_results_csv(condition):
    # condition = "AD"
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(2,10.1,0.1):
            data_frame = Run(SAMPLE_SIZE=100, CONDITION=condition, L_min_w=L_min_w, k_look_w=k_look_w, p_max_w=p_max_w, L_min_s=i, k_look_s=k_look_s, p_max_s=j)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'experiment_1_{condition}.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results

In [ ]:
def exp_1_con_ad_L_k_df_results_csv(condition):
    # condition = "AD"
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(0,5.1,0.1):
            data_frame = Run(SAMPLE_SIZE=100, CONDITION=condition, L_min_w=L_min_w, k_look_w=k_look_w, p_max_w=p_max_w, L_min_s=i, k_look_s=j, p_max_s=p_max_s)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'experiment_1_{condition}.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results

#### Condition AD: Execution

In [ ]:
exp_1_con_ad_df_exp_results = Compare_condition_AD_all_data('AD')

In [ ]:
exp_1_con_ad_p_L_df_exp_results = exp_1_con_ad_p_L_df_results_csv('AD_p_L')

In [ ]:
exp_1_con_ad_L_k_df_exp_results = exp_1_con_ad_L_k_df_results_csv('AD_L_k')

#### Condition AD: Analysis

##### Read from file

In [ ]:
# Read from file
condition = 'AD'
readname = f'experiment_1_{condition}.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
exp_1_con_ad_df_results_csv = pd.read_csv(fullname)

In [ ]:
# Read from file
condition = 'AD_p_L'
readname = f'experiment_1_{condition}.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
exp_1_con_ad_p_l_df_results_csv = pd.read_csv(fullname)

In [ ]:
# Read from file
condition = 'AD_L_k'
readname = f'experiment_1_{condition}.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
exp_1_con_ad_l_k_df_results_csv = pd.read_csv(fullname)

##### L_min_w & L_min_s

In [ ]:
# evaluate L_min_w and L_min_s
con_ad_net_entry_rate_L_min_w_L_min_s = exp_1_con_ad_df_results_csv[['net_entry_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = con_ad_net_entry_rate_L_min_w_L_min_s.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'net_entry_rate')
plt.xlabel('L_min_s', size=15)
plt.ylabel('L_min_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.5,y_stddev=.5)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()


In [ ]:
# evaluate L_min_w and k_look_w
con_ad_net_entry_rate_L_min_w_L_min_s = exp_1_con_ad_df_results_csv[['net_entry_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = con_ad_net_entry_rate_L_min_w_L_min_s.groupby(['L_min_w', 'L_min_s']).std()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'net_entry_rate')
plt.xlabel('L_min_s', size=15)
plt.ylabel('L_min_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.4,y_stddev=.4)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()


##### standard deviation & sentence length

In [ ]:
net_entry_rate = exp_1_con_ad_df_results_csv[['sentence_length_in_words', 'net_entry_rate']]
net_entry_rate_std = net_entry_rate.groupby(['sentence_length_in_words']).mean()
print(net_entry_rate_std)

fig, ax = plt.subplots(figsize=(10,5))
# pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'sentence_length', 'net_entry_rate')
plt.xlabel('sentence_length_in_words', size=15)
plt.ylabel('net_entry_rate', size=15)
plt.plot(net_entry_rate_std, label='net entry rate')
# plt.axhline(y=net_entry_rate_std['net_entry_rate'].mean(), color='k', linestyle='--', label='average of net entry rate')
plt.axvline(x=4.23, color='k', linestyle='--', label='average sentence length')
plt.legend(loc = 'upper right')
plt.show()



In [ ]:
# evaluate L_min_w and k_look_w
con_ad_net_entry_rate_L_min_w_L_min_s = exp_1_con_ad_df_results_csv[['KS_saving_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = con_ad_net_entry_rate_L_min_w_L_min_s.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'KS_saving_rate')
plt.xlabel('L_min_s', size=15)
plt.ylabel('L_min_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.4,y_stddev=.4)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
cb = plt.colorbar()
cb.ax.set_yticklabels(["{:.1%}".format(i) for i in cb.get_ticks()])

In [ ]:
# evaluate L_min_w and k_look_w
con_ad_net_entry_rate_L_min_w_L_min_s = exp_1_con_ad_df_results_csv[['KS_saving_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = con_ad_net_entry_rate_L_min_w_L_min_s.groupby(['L_min_w', 'L_min_s']).std()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'KS_saving_rate')
plt.xlabel('L_min_s', size=15)
plt.ylabel('L_min_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.4,y_stddev=.4)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
cb = plt.colorbar()

##### L_min_s & p_max_s

In [ ]:
# evaluate L_min_w and L_min_s
con_ad_net_entry_rate_L_min_w_L_min_s = exp_1_con_ad_p_l_df_results_csv[['net_entry_rate', 'p_max_s', 'L_min_s']]
vis_net_entry_rate_mean = con_ad_net_entry_rate_L_min_w_L_min_s.groupby(['p_max_s', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('p_max_s', 'L_min_s', 'net_entry_rate')
plt.xlabel('L_min_s', size=15)
plt.ylabel('p_max_s', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.5,y_stddev=.5)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()

##### L & k

In [ ]:
# evaluate k_look_s and L_min_s
con_ad_net_entry_rate_L_min_w_L_min_s = exp_1_con_ad_l_k_df_results_csv[['net_entry_rate', 'L_min_s', 'k_look_s']]
vis_net_entry_rate_mean = con_ad_net_entry_rate_L_min_w_L_min_s.groupby(['L_min_s', 'k_look_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('k_look_s', 'L_min_s', 'net_entry_rate')
plt.xlabel('L_min_s', size=15)
plt.ylabel('k_look_s', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.5,y_stddev=.5)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(0, 5, 6, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()


# # evaluate L_min_s and p_max_s
# con_ac_net_entry_rate_L_min_s_k_look_s = con_ac_df_exp_p_L_results_csv[['net_entry_rate', 'L_min_s', 'p_max_s']]
# vis_net_entry_rate_mean = con_ac_net_entry_rate_L_min_s_k_look_s.groupby(['L_min_s', 'p_max_s']).mean()
# vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

# fig, ax = plt.subplots(figsize=(10,5))
# pivot_table = vis_net_entry_rate_mean.pivot('p_max_s', 'L_min_s', 'net_entry_rate')
# plt.xlabel('L_min_s', size=15)
# plt.ylabel('p_max_s', size=15)
# plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# # plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.4,y_stddev=.4)), cmap='coolwarm', interpolation='nearest')
# ax.invert_yaxis()
# ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
# xticks = np.linspace(2, 10, 9, dtype=np.int16)
# ax.set_xticklabels(xticks)
# ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
# yticks = np.linspace(1, 6, 6, dtype=np.int16)
# ax.set_yticklabels(yticks)
# plt.colorbar()

##### net_entry_rate & sentence_length_in_words

In [ ]:
# evaluate net_entry_rate and sentence_length_in_words
con_ad_net_entry_rate_L_min_s_k_look_s = exp_1_con_ad_df_results_csv[['net_entry_rate', 'L_min_s', 'sentence_length_in_words']] # _in_words
con_ad_net_entry_rate_mean_L_min_s = con_ad_net_entry_rate_L_min_s_k_look_s.groupby(['L_min_s', 'sentence_length_in_words']).mean()
vis_net_entry_rate_mean = con_ad_net_entry_rate_mean_L_min_s.groupby(['sentence_length_in_words']).mean()
# vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
# pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'sentence_length', 'net_entry_rate')
plt.xlabel('sentence_length_in_words', size=15)
plt.ylabel('net_entry_rate_mean', size=15)
plt.plot(vis_net_entry_rate_mean)
plt.axhline(y=0, color='k', linestyle='--')
plt.show()


##### net_entry_rate & sentence_length_in_words std

In [ ]:
# evaluate net_entry_rate and sentence_length_in_words
con_ad_net_entry_rate_L_min_s_k_look_s = exp_1_con_ad_df_results_csv[['net_entry_rate', 'L_min_s', 'sentence_length_in_words']] # _in_words
con_ad_net_entry_rate_mean_L_min_s = con_ad_net_entry_rate_L_min_s_k_look_s.groupby(['L_min_s', 'sentence_length_in_words']).mean()
vis_net_entry_rate_mean = con_ad_net_entry_rate_mean_L_min_s.groupby(['sentence_length_in_words']).std()
# vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
# pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'sentence_length', 'net_entry_rate')
plt.xlabel('sentence_length_in_words', size=15)
plt.ylabel('net_entry_rate_mean', size=15)
plt.plot(vis_net_entry_rate_mean)
plt.axhline(y=0, color='k', linestyle='--')
plt.show()


##### KS_saving_rate & sentence_length_in_words

In [ ]:
# evaluate KS_saving_rate and sentence_length_in_words
con_ad_KS_saving_rate_L_min_s_k_look_s = exp_1_con_ad_df_results_csv[['KS_saving_rate', 'L_min_s', 'sentence_length_in_words']]
con_ad_KS_saving_rate_mean_L_min_s = con_ad_KS_saving_rate_L_min_s_k_look_s.groupby(['L_min_s', 'sentence_length_in_words']).mean()
vis_KS_saving_rate_mean = con_ad_KS_saving_rate_mean_L_min_s.groupby(['sentence_length_in_words']).mean()
# vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
# pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'sentence_length', 'net_entry_rate')
plt.xlabel('sentence_length_in_words', size=15)
plt.ylabel('KS_saving_rate_mean', size=15)
plt.plot(vis_KS_saving_rate_mean, label="KS saving rate")
KS_saving_rate_mean_mean = vis_KS_saving_rate_mean['KS_saving_rate'].mean()
# print(vis_KS_saving_rate_mean)
plt.axhline(y=KS_saving_rate_mean_mean, color='k', linestyle='--', label='average KS saving rate')
plt.legend(loc = 'upper right')

vals = ax.get_yticks()
ax.set_yticklabels(['{:,.0%}'.format(x) for x in vals])

plt.show()

##### KS_saving_rate & sentence_length

In [ ]:
# evaluate KS_saving_rate and sentence_length_in_words
con_ad_KS_saving_rate_L_min_s_k_look_s = exp_1_con_ad_df_results_csv[['KS_saving_rate', 'L_min_s', 'sentence_length']]
con_ad_KS_saving_rate_mean_L_min_s = con_ad_KS_saving_rate_L_min_s_k_look_s.groupby(['L_min_s', 'sentence_length']).mean()
vis_KS_saving_rate_mean = con_ad_KS_saving_rate_mean_L_min_s.groupby(['sentence_length']).mean()
# vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
# pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'sentence_length', 'net_entry_rate')
plt.xlabel('sentence_length', size=15)
plt.ylabel('KS_saving_rate_mean (%)', size=15)
plt.plot(vis_KS_saving_rate_mean, label="KS saving rate")
KS_saving_rate_mean_mean = vis_KS_saving_rate_mean['KS_saving_rate'].mean()
# print(vis_KS_saving_rate_mean)
plt.axhline(y=KS_saving_rate_mean_mean, color='k', linestyle='--', label='average KS saving rate')
plt.legend(loc = 'upper right')
vals = ax.get_yticks()
ax.set_yticklabels(['{:,.0%}'.format(x) for x in vals])
plt.show()

#### When sentence_length_in_words = 6

In [ ]:
con_ad_sen_len_eq_6 = exp_1_con_ad_df_results_csv.loc[exp_1_con_ad_df_results_csv['sentence_length_in_words']==6]
# print(con_ad_sen_len_eq_6)
vis_net_entry_rate_mean = con_ad_net_entry_rate_L_min_w_L_min_s.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'net_entry_rate')
plt.xlabel('L_min_w', size=15)
plt.ylabel('L_min_s', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.4,y_stddev=.4)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()

In [ ]:
con_ad_sen_len_eq_6 = con_ad_df_exp_results_csv.loc[con_ad_df_exp_results_csv['sentence_length_in_words']==6]


### Playground

#### difference between AAC and able-bodied in entry rate

In [ ]:
# User profile: T_aac - T_able
T_key =  0.34 # 
T_react_w = 0.75 # 0.45 # 1.20
T_react_s = 3.39 # T_react_w * L_s_ave = 1.20*4.52 = 5.42

In [ ]:
con_ad_df_exp_results_diff = Compare_condition_AD_all_data('AD-diff')

In [ ]:
# Read from file
condition = 'AD-diff'
readname = f'experiment_1_{condition}.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
con_ad_diff_df_exp_results_csv = pd.read_csv(fullname)

In [ ]:
# evaluate net_entry_rate and sentence_length_in_words
con_ad_net_entry_rate_L_min_s_k_look_s = con_ad_diff_df_exp_results_csv[['entry_rate', 'sentence_length']] # _in_words
vis_net_entry_rate_mean = con_ad_net_entry_rate_L_min_s_k_look_s.groupby(['sentence_length']).mean()
# vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
# pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'sentence_length', 'net_entry_rate')
plt.xlabel('sentence_length (characters)', size=15)
plt.ylabel('entry_rate_diff (Able-bodied_entry_rate - AAC_entry_rate)', size=15)
plt.plot(vis_net_entry_rate_mean)
plt.axhline(y=0, color='k', linestyle='--')
plt.show()

### Condition AD-k: Best word pred settings for sentence pred

#### Load initial parameters and functions

In [ ]:
# Text entering strategy
L_min_w = 7 # 
k_look_w = 0 # 0-5
p_max_w = 6
L_min_s = 5
k_look_s = 0 # 0-5
p_max_s = 8

In [ ]:
def Compare_condition_AD_k_all_data(condition):
    exp_result_dic_list = []
    for i in np.arange(0,5.1,0.1):
        for j in np.arange(0,5.1,0.1):
            data_frame = Run(SAMPLE_SIZE=100, CONDITION=condition, L_min_w=L_min_w, k_look_w=i, p_max_w=p_max_w, L_min_s=L_min_s, k_look_s=j, p_max_s=p_max_s)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'experiment_1_{condition}.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results

In [ ]:
con_ad_k_df_exp_results = Compare_condition_AD_k_all_data('AD_k')

##### Read from file

In [ ]:
# Read from file
condition = 'AD_K'
readname = f'experiment_1_{condition}.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
con_ad_k_df_exp_results_csv = pd.read_csv(fullname)

##### L_min_w & k_look_w

In [ ]:
# evaluate k_look_s and k_look_w
con_ad_k_net_entry_rate_L_min_s_L_min_w = con_ad_k_df_exp_results_csv[['net_entry_rate', 'k_look_s', 'k_look_w']]
vis_net_entry_rate_mean = con_ad_k_net_entry_rate_L_min_s_L_min_w.groupby(['k_look_s', 'k_look_w']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('k_look_w', 'k_look_s', 'net_entry_rate')
plt.xlabel('k_look_s', size=15)
plt.ylabel('k_look_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.4,y_stddev=.4)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(0, 5, 6, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(0, 5, 6, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()


In [ ]:
# evaluate L_min_w and k_look_w
con_ad_k_net_entry_rate_L_min_s_L_min_w = con_ad_k_df_exp_results_csv[['net_entry_rate', 'k_look_s', 'k_look_w']]
vis_net_entry_rate_mean = con_ad_k_net_entry_rate_L_min_s_L_min_w.groupby(['k_look_s', 'k_look_w']).std()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('k_look_w', 'k_look_s', 'net_entry_rate')
plt.xlabel('k_look_s', size=15)
plt.ylabel('k_look_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.4,y_stddev=.4)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(0, 5, 6, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(0, 5, 6, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()


In [ ]:
# evaluate k_look_s and k_look_w
con_ad_k_KS_saving_rate_k_look_s_k_look_w = con_ad_k_df_exp_results_csv[['KS_saving_rate', 'k_look_s', 'k_look_w']]
vis_KS_saving_rate_mean = con_ad_k_KS_saving_rate_k_look_s_k_look_w.groupby(['k_look_s', 'k_look_w']).mean()
vis_KS_saving_rate_mean = vis_KS_saving_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_KS_saving_rate_mean.pivot('k_look_w', 'k_look_s', 'KS_saving_rate')
plt.xlabel('k_look_s', size=15)
plt.ylabel('k_look_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.4,y_stddev=.4)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(0, 5, 6, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(0, 5, 6, dtype=np.int16)
ax.set_yticklabels(yticks)
cb = plt.colorbar()
cb.ax.set_yticklabels(["{:.1%}".format(i) for i in cb.get_ticks()])

## Experiment 2: KLM-BEI

### Load fundamental function

In [ ]:
def Run_exp_2(SAMPLE_SIZE, CONDITION, R_rational, R_error, R_interrupt, T_event, L_min_w=100.0, k_look_w=0.0, p_max_w=0.0, L_min_s=100.0, k_look_s=0.0, p_max_s=0.0):
    with open(f'aac_daily_conversation_sample_{SAMPLE_SIZE}.txt') as file:
        samples = file.readlines()
    # samples = ["Oh so what did dad have to say to you?"]
    data_dic_list = []

    for i, s in enumerate(samples):
        # print(f"In sample {i}/{len(samples)}: {s}")
        sample = s.strip()
        sample = re.sub(r'[^\w\s]', '', sample)
        word_list = sample.split(' ')
        dic_entry_record, T_total, KS_total = Type_one_sentence(sample, L_min_w, k_look_w, p_max_w, L_min_s, k_look_s, p_max_s, R_rational=R_rational, R_error=R_error, R_interrupt=R_interrupt, T_event=T_event) # Take_average_of_one_sentence_result(1, sample)
        data_dic = {
            # settings
            "L_min_w": float(L_min_w),
            "k_look_w": float(k_look_w),
            "p_max_w": float(p_max_w),
            "L_min_s": float(L_min_s),
            "k_look_s": float(k_look_s),
            "p_max_s": float(p_max_s),
            "P_pred_w_c": float(P_pred_w_c),
            "P_pred_w_n": float(P_pred_w_n),
            "T_key": float(T_key),
            "T_react_w": float(T_react_w),
            "T_react_s": float(T_react_s),
            "R_rational": float(R_rational), # bounded rationality index
            "R_error": float(R_error), # human error rate
            "R_interrupt": float(R_interrupt), # interruption rate
            "T_event": float(T_event), # interruption time

            # results
            "sentence": sample,
            "sentence_length": int(len(sample)),
            "sentence_length_in_words": int(len(word_list)),

            "T_total": dic_entry_record['T_total'],
            "T_key_total": dic_entry_record['T_key_total'],
            "T_react_w_total": dic_entry_record['T_react_w_total'],
            "T_react_s_total": dic_entry_record['T_react_s_total'],
            "T_react_total": dic_entry_record['T_react_total'],
            "T_bounded_l_total": dic_entry_record['T_bounded_l_total'],
            "T_bounded_w_total": dic_entry_record['T_bounded_w_total'],
            "T_bounded_s_total": dic_entry_record['T_bounded_s_total'],
            "T_net_extra": dic_entry_record['T_net_extra'],
            "T_error_total": dic_entry_record['T_error_total'],
            "T_interrupt_total": dic_entry_record['T_interrupt_total'],
            "T_event_total": dic_entry_record['T_event_total'],

            "N_bounded_l_total": dic_entry_record['N_bounded_l_total'],
            "N_bounded_w_total": dic_entry_record['N_bounded_w_total'], 
            "N_bounded_s_total": dic_entry_record['N_bounded_s_total'], 
            "N_bounded_total": dic_entry_record['N_bounded_total'],

            "KS_total": dic_entry_record['KS_total'],
            "KS_type_total": dic_entry_record['KS_type_total'],
            "KS_select_total": dic_entry_record['KS_select_total'],
            "KS_bounded_l_total": dic_entry_record['KS_bounded_l_total'],
            "KS_bounded_w_total": dic_entry_record['KS_bounded_w_total'],
            "KS_bounded_s_total": dic_entry_record['KS_bounded_s_total'],
            "KS_error_total": dic_entry_record['KS_error_total'],
            "KS_interrupt_total": dic_entry_record['KS_interrupt_total'],

            # basic statistics
            "entry_rate": float(len(sample)/T_total)*12.0, # wpm
            # "effective_entry_rate":
            "net_entry_rate": float((len(sample)/T_total)*12.0 - (12.0/T_key)), # wpm
            # "net_effective_entry_rate":
            "KS_saving": float(float(len(sample)) - KS_total),
            "KS_saving_rate": float(float(len(sample)) - KS_total)/float(len(sample)) # %
        }

        data_dic_list.append(data_dic)

    # data_dic_list.sort(key=sort_by_sentence_length)

    # Collect data
    df_data_dic_list = pd.DataFrame(data=data_dic_list)

    return df_data_dic_list

### Load fixed parameters for AAC users

In [ ]:
# User profile
T_key =  0.6 # 0.26 # 0.6
T_react_w = 1.20 # 0.45 # 1.20
T_react_s = 5.08 # 1.90 # T_react_w * L_s_ave = 1.20*4.23 = 5.08

# Word prediction settings
P_pred_w_c = 0.6 # KWickChat
P_pred_w_n = 0.5 # KWickChat

# Sentence prediction settings
def P_pred_s(current_word_index, L_s): # the probability of the system have correct sentence predictions
    a=1
    b=L_s
    x=current_word_index
    c=0
    return a*math.log(x+1,b)+c

# Text entering strategy
L_min_w = 5 # 2-10
k_look_w = 1
p_max_w = 6

L_min_s = 5 # 2-10
k_look_s = 1
p_max_s = 6


### Condition AD - exp_2: L_min_w & L_min_s with BE

#### Condition AD: Execution

In [ ]:
# Human factors
R_rational = 0.8 # 0.1~1
R_error = 0.3 # 0~0.5
R_interrupt = 0.2

T_event = 5

def T_interrupt(T_event): # the probability of the system have correct sentence predictions
    a=0.189
    c=1.03
    return a*math.log(T_event)+c

In [ ]:
def Compare_exp_2_condition_AD_all_data(condition):
    # condition = "AD"
    exp_result_dic_list = []
    for i in np.arange(0,1.05,0.05):
        for j in np.arange(0,1.05,0.05):
            data_frame = Run_exp_2(SAMPLE_SIZE=100, CONDITION=condition, R_rational=i, R_error=j, R_interrupt=0.2, T_event=5, L_min_w=L_min_w, k_look_w=k_look_w, p_max_w=p_max_w, L_min_s=L_min_s, k_look_s=k_look_s, p_max_s=p_max_s)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'experiment_2_{condition}.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results

In [ ]:
def Compare_exp_2_condition_AD_ideal(condition):
    # condition = "AD"
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(2,10.1,0.1):
            data_frame = Run_exp_2(SAMPLE_SIZE=100, CONDITION=condition, R_rational=R_rational, R_error=R_error, R_interrupt=R_interrupt, T_event=5, L_min_w=i, k_look_w=k_look_w, p_max_w=p_max_w, L_min_s=j, k_look_s=k_look_s, p_max_s=p_max_s)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'experiment_2_{condition}.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results

In [ ]:
BOOL_PRINT = False
con_ad_df_exp_results = Compare_exp_2_condition_AD_all_data('best_AD_BEI')

In [ ]:
R_rational = 1
R_error = 0
R_interrupt = 0

BOOL_PRINT = False
con_ad_df_exp_results_ideal = Compare_exp_2_condition_AD_ideal('AD_BEI_ideal')

#### Condition AD: Analysis

##### Read from file

In [ ]:
# Read from file
condition = 'best_AD_BEI'
readname = f'experiment_2_{condition}.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
exp_2_con_ad_df_results_bei_csv = pd.read_csv(fullname)

In [ ]:
# Read from file
condition = 'AD_BEI_ideal'
readname = f'experiment_2_{condition}.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
exp_2_con_ad_df_results_bei_ideal_csv = pd.read_csv(fullname)

##### R_rational & R_error

In [ ]:
entry_rate = exp_2_con_ad_df_results_bei_csv[['net_entry_rate', 'R_rational', 'R_error']]

entry_rate_mean = entry_rate.groupby(['R_rational', 'R_error']).mean()
entry_rate_mean = entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = entry_rate_mean.pivot('R_rational', 'R_error', 'net_entry_rate')
plt.xlabel('R_error (%)', size=15)
plt.ylabel('R_rational (%)', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.4,y_stddev=.4)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,5))
xticks = np.linspace(0, 100, 5, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,5))
yticks = np.linspace(0, 100, 5, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()

# Save data
outname = f'exp2_B_E_ner.csv'
outdir = './experiment_result'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
pivot_table.to_csv(fullname)


In [ ]:
entry_rate = exp_2_con_ad_df_results_bei_csv[['KS_saving_rate', 'R_rational', 'R_error']]

entry_rate_mean = entry_rate.groupby(['R_rational', 'R_error']).mean()
entry_rate_mean = entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = entry_rate_mean.pivot('R_rational', 'R_error', 'KS_saving_rate')
plt.xlabel('R_error (%)', size=15)
plt.ylabel('R_rational (%)', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.4,y_stddev=.4)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,5))
xticks = np.linspace(0, 100, 5, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,5))
yticks = np.linspace(0, 100, 5, dtype=np.int16)
ax.set_yticklabels(yticks)
cbar = plt.colorbar()
cbar.ax.yaxis.set_major_formatter(PercentFormatter(1, 0))

# # Save data
# outname = f'exp2_B_E_ner.csv'
# outdir = './experiment_result'
# if not os.path.exists(outdir):
#     os.mkdir(outdir)
# fullname = os.path.join(outdir, outname) 
# pivot_table.to_csv(fullname)

##### L_min_s & L_min_w

In [ ]:
# evaluate L_min_w and k_look_w

entry_rate = exp_2_con_ad_df_results_bei_ideal_csv[['net_entry_rate', 'L_min_w', 'L_min_s']]
entry_rate_mean = entry_rate.groupby(['L_min_w', 'L_min_s']).mean()
entry_rate_mean = entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = entry_rate_mean.pivot('L_min_w', 'L_min_s', 'net_entry_rate')
plt.xlabel('L_min_s', size=15)
plt.ylabel('L_min_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.5,y_stddev=.5)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()


In [ ]:
# evaluate L_min_w and k_look_w
entry_rate = exp_1_con_ad_df_results_csv[['net_entry_rate', 'L_min_w', 'L_min_s']]
entry_rate_mean = entry_rate.groupby(['L_min_w', 'L_min_s']).std()
entry_rate_mean = entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = entry_rate_mean.pivot('L_min_w', 'L_min_s', 'net_entry_rate')
plt.xlabel('L_min_w', size=15)
plt.ylabel('L_min_s', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.4,y_stddev=.4)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()


In [ ]:
# evaluate L_min_w and L_min_s
KS_saving_rate = exp_1_con_ad_df_results_csv[['KS_saving_rate', 'L_min_w', 'L_min_s']]
KS_saving_rate_mean = KS_saving_rate.groupby(['L_min_w', 'L_min_s']).mean()
KS_saving_rate_mean = KS_saving_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = KS_saving_rate_mean.pivot('L_min_w', 'L_min_s', 'KS_saving_rate')
plt.xlabel('L_min_w', size=15)
plt.ylabel('L_min_s', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.4,y_stddev=.4)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
cb = plt.colorbar()
cb.ax.set_yticklabels(["{:.1%}".format(i) for i in cb.get_ticks()])

In [ ]:
# evaluate L_min_w and L_min_s
KS_saving_rate = exp_1_con_ad_df_results_csv[['KS_saving_rate', 'L_min_w', 'L_min_s']]
KS_saving_rate_mean = KS_saving_rate.groupby(['L_min_w', 'L_min_s']).std()
KS_saving_rate_mean = KS_saving_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = KS_saving_rate_mean.pivot('L_min_w', 'L_min_s', 'KS_saving_rate')
plt.xlabel('L_min_w', size=15)
plt.ylabel('L_min_s', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.4,y_stddev=.4)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
cb = plt.colorbar()
# cb.ax.set_yticklabels(["{:.1%}".format(i) for i in cb.get_ticks()])

##### net_entry_rate & sentence_length_in_words

In [ ]:
# evaluate net_entry_rate and sentence_length_in_words
con_ad_net_entry_rate_L_min_s_k_look_s = exp_1_con_ad_df_results_csv[['net_entry_rate', 'L_min_s', 'sentence_length_in_words']] # _in_words
con_ad_net_entry_rate_mean_L_min_s = con_ad_net_entry_rate_L_min_s_k_look_s.groupby(['L_min_s', 'sentence_length_in_words']).mean()
vis_net_entry_rate_mean = con_ad_net_entry_rate_mean_L_min_s.groupby(['sentence_length_in_words']).mean()
# vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
# pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'sentence_length', 'net_entry_rate')
plt.xlabel('sentence_length_in_words', size=15)
plt.ylabel('net_entry_rate_mean', size=15)
plt.plot(vis_net_entry_rate_mean)
plt.axhline(y=0, color='k', linestyle='--')
plt.show()


#### Compare difference between KLM and KLM-BE

In [ ]:
exp_1_entry_rate = exp_1_con_ad_df_results_csv[['L_min_w', 'L_min_s', 'entry_rate']]
exp_2_entry_rate = exp_2_con_ad_df_results_csv[['L_min_w', 'L_min_s', 'entry_rate']]

diff_entry_rate = exp_1_entry_rate.rename(columns={'entry_rate': 'entry_rate_1'})
diff_entry_rate['entry_rate_2'] = exp_2_entry_rate['entry_rate']
diff_entry_rate['entry_rate_diff'] = diff_entry_rate['entry_rate_2'].subtract(diff_entry_rate['entry_rate_1'])
diff_entry_rate = diff_entry_rate[['L_min_w', 'L_min_s', 'entry_rate_diff']]

diff_entry_rate_mean = diff_entry_rate.groupby(['L_min_w', 'L_min_s']).mean()
diff_entry_rate_mean = diff_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = diff_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'entry_rate_diff')
plt.xlabel('L_min_w', size=15)
plt.ylabel('L_min_s', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.4,y_stddev=.4)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()

diff_entry_rate_std = diff_entry_rate.groupby(['L_min_w', 'L_min_s']).std()
diff_entry_rate_std = diff_entry_rate_std.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = diff_entry_rate_std.pivot('L_min_w', 'L_min_s', 'entry_rate_diff')
plt.xlabel('L_min_w', size=15)
plt.ylabel('L_min_s', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.4,y_stddev=.4)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()


# print(diff_entry_rate)

How R_rational influence the entry rate

In [ ]:
# def sort_by_sentence_length(e):
#     return e["sentence_length_in_words"]
    
def Run_exp_3(SAMPLE_SIZE, CONDITION, L_min_w=100.0, k_look_w=0.0, p_max_w=0.0, L_min_s=100.0, k_look_s=0.0, p_max_s=0.0):
    with open(f'aac_daily_conversation_sample_{SAMPLE_SIZE}.txt') as file:
        samples = file.readlines()
    # samples = ["Oh so what did dad have to say to you?"]
    data_dic_list = []

    for i, s in enumerate(samples):
        # print(f"In sample {i}/{len(samples)}: {s}")
        sample = s.strip()
        sample = re.sub(r'[^\w\s]', '', sample)
        word_list = sample.split(' ')
        dic_entry_record, T_total, KS_total = Type_one_sentence(sample, L_min_w, k_look_w, p_max_w, L_min_s, k_look_s, p_max_s) # Take_average_of_one_sentence_result(1, sample)
        data_dic = {
            # settings
            "L_min_w": float(L_min_w),
            "k_look_w": float(k_look_w),
            "p_max_w": float(p_max_w),
            "L_min_s": float(L_min_s),
            "k_look_s": float(k_look_s),
            "p_max_s": float(p_max_s),
            "P_pred_w_c": float(P_pred_w_c),
            "P_pred_w_n": float(P_pred_w_n),
            "T_key": float(T_key),
            "T_react_w": float(T_react_w),
            "T_react_s": float(T_react_s),
            "R_rational": float(R_rational), # bounded rationality index
            "R_error": float(R_error), # human error rate
            "R_interrupt": float(R_interrupt), # interruption rate
            "T_event": float(T_event), # interruption time

            # results
            "sentence": sample,
            "sentence_length": int(len(sample)),
            "sentence_length_in_words": int(len(word_list)),

            "T_total": dic_entry_record['T_total'],
            "T_key_total": dic_entry_record['T_key_total'],
            "T_react_w_total": dic_entry_record['T_react_w_total'],
            "T_react_s_total": dic_entry_record['T_react_s_total'],
            "T_react_total": dic_entry_record['T_react_total'],
            "T_bounded_l_total": dic_entry_record['T_bounded_l_total'],
            "T_bounded_w_total": dic_entry_record['T_bounded_w_total'],
            "T_bounded_s_total": dic_entry_record['T_bounded_s_total'],
            "T_error_total": dic_entry_record['T_error_total'],
            "T_interrupt_total": dic_entry_record['T_interrupt_total'],

            "T_event_total": dic_entry_record['T_event_total'],
            "N_bounded_l_total": dic_entry_record['N_bounded_l_total'],
            "N_bounded_w_total": dic_entry_record['N_bounded_w_total'], 
            "N_bounded_s_total": dic_entry_record['N_bounded_s_total'], 
            "N_bounded_total": dic_entry_record['N_bounded_total'],

            "KS_total": dic_entry_record['KS_total'],
            "KS_type_total": dic_entry_record['KS_type_total'],
            "KS_select_total": dic_entry_record['KS_select_total'],
            "KS_bounded_l_total": dic_entry_record['KS_bounded_l_total'],
            "KS_bounded_w_total": dic_entry_record['KS_bounded_w_total'],
            "KS_bounded_s_total": dic_entry_record['KS_bounded_s_total'],
            "KS_error_total": dic_entry_record['KS_error_total'],
            "KS_interrupt_total": dic_entry_record['KS_interrupt_total'],

            # basic statistics
            "entry_rate": float(len(sample)/T_total)*12.0, # wpm
            # "effective_entry_rate":
            "net_entry_rate": float((len(sample)/T_total)*12.0 - (12.0/T_key)), # wpm
            # "net_effective_entry_rate":
            "KS_saving": float(float(len(sample)) - KS_total),
            "KS_saving_rate": float(float(len(sample)) - KS_total)/float(len(sample)) # %
        }

        data_dic_list.append(data_dic)

    # data_dic_list.sort(key=sort_by_sentence_length)

    # Collect data
    df_data_dic_list = pd.DataFrame(data=data_dic_list)

    return df_data_dic_list

## Experiment 3: Simulate imperfect user

### Load fundamental function

In [ ]:
def Run_exp_3(SAMPLE_SIZE, CONDITION, L_min_w=100.0, k_look_w=0.0, p_max_w=0.0, L_min_s=100.0, k_look_s=0.0, p_max_s=0.0, R_rational=1.0, R_error=0.0, R_interrupt=0.0, T_event=5.0):
    with open(f'aac_daily_conversation_sample_{SAMPLE_SIZE}.txt') as file:
        samples = file.readlines()
    # samples = ["Oh so what did dad have to say to you?"]
    data_dic_list = []

    for i, s in enumerate(samples):
        # print(f"In sample {i}/{len(samples)}: {s}")
        sample = s.strip()
        sample = re.sub(r'[^\w\s]', '', sample)
        word_list = sample.split(' ')
        dic_entry_record, T_total, KS_total = Type_one_sentence(sample, L_min_w, k_look_w, p_max_w, L_min_s, k_look_s, p_max_s, R_rational, R_error, R_interrupt, T_event) # Take_average_of_one_sentence_result(1, sample)

        data_dic = {
            # settings
            "L_min_w": float(L_min_w),
            "k_look_w": float(k_look_w),
            "p_max_w": float(p_max_w),
            "L_min_s": float(L_min_s),
            "k_look_s": float(k_look_s),
            "p_max_s": float(p_max_s),
            "P_pred_w_c": float(P_pred_w_c),
            "P_pred_w_n": float(P_pred_w_n),
            "T_key": float(T_key),
            "T_react_w": float(T_react_w),
            "T_react_s": float(T_react_s),
            "R_rational": float(R_rational), # bounded rationality index
            "R_error": float(R_error), # human error rate
            "R_interrupt": float(R_interrupt), # interruption rate
            "T_event": float(T_event), # interruption time

            # results
            "sentence": sample,
            "sentence_length": int(len(sample)),
            "sentence_length_in_words": int(len(word_list)),

            # T: 
            "T_total": dic_entry_record['T_total'],
            "T_key_total": dic_entry_record['T_key_total'],
            "T_react_w_total": dic_entry_record['T_react_w_total'],
            "T_react_s_total": dic_entry_record['T_react_s_total'],
            "T_react_total": dic_entry_record['T_react_total'],
            # bounded rationality
            "T_extra_total": dic_entry_record['T_extra_total'],
            "T_extra_s": dic_entry_record['T_extra_s'],
            "T_extra_w": dic_entry_record['T_extra_w'],
            "T_extra_l": dic_entry_record['T_extra_l'],
            "T_net_extra": dic_entry_record['T_net_extra'],
            # human error
            "T_error_total": dic_entry_record['T_error_total'],       
            # interruption
            "T_interrupt_total": dic_entry_record['T_interrupt_total'],
            "T_event_total": dic_entry_record['T_event_total'],

            # KS: 
            "KS_total": dic_entry_record['KS_total'],
            "KS_type_total": dic_entry_record['KS_type_total'],
            "KS_select_total": dic_entry_record['KS_select_total'],
            # bounded rationality
            "KS_extra_total": dic_entry_record['KS_extra_total'],
            "KS_extra_s": dic_entry_record['KS_extra_s'],
            "KS_extra_w": dic_entry_record['KS_extra_w'],
            "KS_extra_l": dic_entry_record['KS_extra_l'],
            # human error
            "KS_error_total": dic_entry_record['KS_error_total'],
            # interruption no KS

            # basic statistics
            "entry_rate": float(len(sample)/T_total)*12.0, # wpm
            "net_entry_rate": float((len(sample)/T_total)*12.0 - (12.0/T_key)), # wpm
            "KS_saving": float(float(len(sample)) - KS_total),
            "KS_saving_rate": float(float(len(sample)) - KS_total)/float(len(sample)), # %

            # Human performance analyses
            "effective_typing_rate_index": float(  ((dic_entry_record['KS_total']-dic_entry_record['KS_error_total'])/dic_entry_record['T_key_total']) * T_key ),
            "error_index": float(dic_entry_record['KS_error_total']/(dic_entry_record['KS_total']-dic_entry_record['KS_extra_total'])),
            "rational_index": float(1- ( dic_entry_record['KS_extra_total']/(dic_entry_record['KS_total']-dic_entry_record['KS_error_total']) ) ),
            "intuitiveness_index_KS": float(1- ( (dic_entry_record['KS_error_total']+dic_entry_record['KS_extra_total'])/dic_entry_record['KS_total'] ) ),
            "intuitiveness_index_T": float(1- ( (dic_entry_record['T_error_total']+dic_entry_record['T_net_extra'])/dic_entry_record['T_total'] ) )
        }

        data_dic_list.append(data_dic)

    # data_dic_list.sort(key=sort_by_sentence_length)

    # Collect data
    df_data_dic_list = pd.DataFrame(data=data_dic_list)

    return df_data_dic_list

### Load fixed parameters

In [ ]:
# User profile
T_key =  0.26 # 0.26 # 0.6
T_react_w = 0.45 # 0.45 # 1.20
T_react_s = 1.90 # T_react_w * L_s_ave = 1.20*4.23 = 5.08

# Word prediction settings
P_pred_w_c = 0.6 # 0.6 # KWickChat
P_pred_w_n = 0.5 # 0.5 # KWickChat

# Sentence prediction settings
def P_pred_s(current_word_index, L_s): # the probability of the system have correct sentence predictions
    a=1
    b=L_s
    x=current_word_index
    c=0
    return a*math.log(x+1,b)+c

L_min_w = 5
k_look_w = 1 
p_max_w = 6 
L_min_s = 5 
k_look_s = 1 
p_max_s = 6

# Human factors
R_rational = 1 #0.7 # 0.1~1
R_error = 0 #0.4 # 0~0.5
R_interrupt = 0 #0.2

T_event = 5


### Execution: L_min_w & L_min_s

In [ ]:
def Simulate_imperfect_user(condition):
    # condition = "AD"
    exp_result_dic_list = []
    for i in np.arange(2,10.1,0.1):
        for j in np.arange(2,10.1,0.1):
            data_frame = Run_exp_3(SAMPLE_SIZE=100, CONDITION=condition, R_rational=R_rational, R_error=R_error, R_interrupt=R_interrupt, T_event=T_event, L_min_w=i, k_look_w=k_look_w, p_max_w=p_max_w, L_min_s=j, k_look_s=k_look_s, p_max_s=p_max_s)
            exp_result_dic_list.append(data_frame)
            
    exp_results = [df for df in exp_result_dic_list]
    df_exp_results = pd.concat(exp_results, ignore_index=True)

    # Save data
    outname = f'experiment_3_{condition}.csv'
    outdir = './experiment_result'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df_exp_results.to_csv(fullname)
            
    return df_exp_results

In [ ]:
# ideal: Human factors
R_rational = 1 #0.7 # 0.1~1
R_error = 0 #0.3 # 0~0.5
R_interrupt = 0

BOOL_PRINT = False
exp_3_con_ad_df_exp_results_ideal = Simulate_imperfect_user('AD_ideal')

In [ ]:
R_rational = 1 #0.7 # 0.1~1
R_error = 0 #0.3 # 0~0.5
R_interrupt = 0.5 

T_event = 5

BOOL_PRINT = False
exp_3_con_ad_df_exp_results_surrogate = Simulate_imperfect_user('AD_surrogate')

### Analysis

##### Read from file

In [ ]:
# Read from file
condition = 'AD_ideal'
readname = f'experiment_3_{condition}.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
exp_3_con_ad_df_results_ideal_csv = pd.read_csv(fullname)

In [ ]:
# Read from file
condition = 'AD_surrogate'
readname = f'experiment_3_{condition}.csv'
readdir = './experiment_result'
fullname = os.path.join(readdir, readname)
exp_3_con_ad_df_results_surrogate_csv = pd.read_csv(fullname)

#### L_min_w & L_min_s

In [ ]:
# evaluate L_min_w and L_min_s
con_ad_net_entry_rate_L_min_w_L_min_s = exp_3_con_ad_df_results_surrogate_csv[['effective_typing_rate_index', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = con_ad_net_entry_rate_L_min_w_L_min_s.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'effective_typing_rate_index')
plt.xlabel('L_min_s', size=15)
plt.ylabel('L_min_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.5,y_stddev=.5)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
cb = plt.colorbar()
cb.ax.set_yticklabels(["{:.1%}".format(i) for i in cb.get_ticks()])

In [ ]:
# ideal: effective_typing_rate_index
con_ad_net_entry_rate_L_min_w_L_min_s = exp_3_con_ad_df_results_ideal_csv[['effective_typing_rate_index', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = con_ad_net_entry_rate_L_min_w_L_min_s.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'effective_typing_rate_index')
plt.xlabel('L_min_s', size=15)
plt.ylabel('L_min_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.5,y_stddev=.5)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
cb = plt.colorbar()
cb.ax.set_yticklabels(["{:.1%}".format(i) for i in cb.get_ticks()])

In [ ]:
# evaluate L_min_w and L_min_s
con_ad_net_entry_rate_L_min_w_L_min_s = exp_3_con_ad_df_results_surrogate_csv[['effective_typing_rate_index', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = con_ad_net_entry_rate_L_min_w_L_min_s.groupby(['L_min_w', 'L_min_s']).std()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'effective_typing_rate_index')
plt.xlabel('L_min_s', size=15)
plt.ylabel('L_min_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.5,y_stddev=.5)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
cb = plt.colorbar()
cb.ax.set_yticklabels(["{:.1%}".format(i) for i in cb.get_ticks()])

In [ ]:
# evaluate L_min_w and L_min_s
con_ad_net_entry_rate_L_min_w_L_min_s = exp_3_con_ad_df_results_surrogate_csv[['net_entry_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = con_ad_net_entry_rate_L_min_w_L_min_s.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'net_entry_rate')
plt.xlabel('L_min_s', size=15)
plt.ylabel('L_min_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.5,y_stddev=.5)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()

In [ ]:
# ideal: net_entry_rate
con_ad_net_entry_rate_L_min_w_L_min_s = exp_3_con_ad_df_results_ideal_csv[['net_entry_rate', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = con_ad_net_entry_rate_L_min_w_L_min_s.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'net_entry_rate')
plt.xlabel('L_min_s', size=15)
plt.ylabel('L_min_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.5,y_stddev=.5)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
cb = plt.colorbar()

In [ ]:
# KS_error_total: L_min_w and L_min_s
KS_error_total = exp_3_con_ad_df_results_surrogate_csv[['KS_error_total', 'L_min_w', 'L_min_s']]
KS_error_total_mean = KS_error_total.groupby(['L_min_w', 'L_min_s']).mean()
KS_error_total_mean = KS_error_total_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = KS_error_total_mean.pivot('L_min_w', 'L_min_s', 'KS_error_total')
plt.xlabel('L_min_s', size=15)
plt.ylabel('L_min_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.5,y_stddev=.5)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()

In [ ]:
# KS_extra_total: L_min_w and L_min_s
KS_extra_total = exp_3_con_ad_df_results_surrogate_csv[['KS_extra_total', 'L_min_w', 'L_min_s']]
KS_extra_total_mean = KS_extra_total.groupby(['L_min_w', 'L_min_s']).mean()
KS_extra_total_mean = KS_extra_total_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = KS_extra_total_mean.pivot('L_min_w', 'L_min_s', 'KS_extra_total')
plt.xlabel('L_min_s', size=15)
plt.ylabel('L_min_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.5,y_stddev=.5)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()

In [ ]:
# T_net_extra: L_min_w and L_min_s
con_ad_net_entry_rate_L_min_w_L_min_s = exp_3_con_ad_df_results_surrogate_csv[['T_net_extra', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = con_ad_net_entry_rate_L_min_w_L_min_s.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'T_net_extra')
plt.xlabel('L_min_s', size=15)
plt.ylabel('L_min_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.5,y_stddev=.5)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
plt.colorbar()

##### Rationality index

In [ ]:
# rational_index: L_min_w and L_min_s
con_ad_net_entry_rate_L_min_w_L_min_s = exp_3_con_ad_df_results_surrogate_csv[['rational_index', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = con_ad_net_entry_rate_L_min_w_L_min_s.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'rational_index')
plt.xlabel('L_min_s', size=15)
plt.ylabel('L_min_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.5,y_stddev=.5)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
cb = plt.colorbar()
cb.ax.set_yticklabels(["{:.1%}".format(i) for i in cb.get_ticks()])

##### Error index

In [ ]:
# error_index: L_min_w and L_min_s
con_ad_net_entry_rate_L_min_w_L_min_s = exp_3_con_ad_df_results_surrogate_csv[['error_index', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = con_ad_net_entry_rate_L_min_w_L_min_s.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'error_index')
plt.xlabel('L_min_s', size=15)
plt.ylabel('L_min_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.5,y_stddev=.5)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
cb = plt.colorbar()
cb.ax.set_yticklabels(["{:.1%}".format(i) for i in cb.get_ticks()])

##### Intuitive index KS

In [ ]:
# evaluate L_min_w and L_min_s
con_ad_net_entry_rate_L_min_w_L_min_s = exp_3_con_ad_df_results_surrogate_csv[['intuitiveness_index_KS', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = con_ad_net_entry_rate_L_min_w_L_min_s.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'intuitiveness_index_KS')
plt.xlabel('L_min_s', size=15)
plt.ylabel('L_min_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.5,y_stddev=.5)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
cb = plt.colorbar()
cb.ax.set_yticklabels(["{:.1%}".format(i) for i in cb.get_ticks()])

##### Intuitive index T

In [ ]:
# evaluate L_min_w and L_min_s
con_ad_net_entry_rate_L_min_w_L_min_s = exp_3_con_ad_df_results_surrogate_csv[['intuitiveness_index_T', 'L_min_w', 'L_min_s']]
vis_net_entry_rate_mean = con_ad_net_entry_rate_L_min_w_L_min_s.groupby(['L_min_w', 'L_min_s']).mean()
vis_net_entry_rate_mean = vis_net_entry_rate_mean.reset_index()

fig, ax = plt.subplots(figsize=(10,5))
pivot_table = vis_net_entry_rate_mean.pivot('L_min_w', 'L_min_s', 'intuitiveness_index_T')
plt.xlabel('L_min_s', size=15)
plt.ylabel('L_min_w', size=15)
plt.imshow(pivot_table, cmap='coolwarm', interpolation='nearest')
# plt.imshow(convolve(pivot_table, Gaussian2DKernel(x_stddev=.5,y_stddev=.5)), cmap='coolwarm', interpolation='nearest')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,pivot_table.columns.size+1,10))
xticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,pivot_table.index.size+1,10))
yticks = np.linspace(2, 10, 9, dtype=np.int16)
ax.set_yticklabels(yticks)
cb = plt.colorbar()
cb.ax.set_yticklabels(["{:.1%}".format(i) for i in cb.get_ticks()])